# Perchance 規則轉 Cloudflare D1 SQL 轉換器

這個 Notebook 用於將 Perchance 風格的純文字設定檔，轉換為 Cloudflare D1 資料庫可執行的 SQL `INSERT` 指令。

### 功能：
1. 解析 `Config` 區塊（定義類別、群組、必填/選填）。
2. 解析 `Content` 區塊（定義具體的 Prompt 與中文標籤）。
3. 自動處理單引號跳脫（Escape Single Quotes）。
4. 生成完整的 SQL 腳本。

In [9]:
import re

# ==========================================
# 1.在此貼上你的設定資料 (CONFIG)
# 格式: Slug | Label | Tags
# ==========================================
RAW_CONFIG = """
畫風 | 畫風 | group:Base
體型 | 體型 | group:Base
場景 | 場景 | group:character
特別場景 | 特別場景 | group:character opt
服裝搭配 | 穿搭 | group:character
服裝設計 | 服設 | group:character
胸部姿勢 | 胸姿 | group:pose
臀部姿勢 | 臀姿 | group:pose
腿部姿勢 | 腿姿 | group:pose
體位姿勢 | 體姿 | group:pose
臉型 | 臉型 |  group:Look
髮型 | 髮型 |  group:Look
髮色 | 髮色 |  group:Look opt
表情 | 表情 | group:Look opt
基本配件 | 基本配件 | mul:1 group:Decorations opt
頭配件 | 頭 | mul:1 group:Decorations opt
上半配件 | 上半 | mul:1 group:Decorations opt
下半配件 | 下半 | group:Decorations opt
全身配件 | 全身 | group:Decorations opt
特級配件 | 特級 | group:Decorations opt
瞳色 | 瞳色 | group:FaceDetail opt
虹膜 | 虹膜 | group:FaceDetail opt
眼睫毛 | 眼睫毛 | group:FaceDetail opt
眼妝 | 眼妝 | group:FaceDetail opt
唇狀 | 唇狀 | group:FaceDetail opt
唇色 | 唇色 | group:FaceDetail opt
唇感 | 唇感 | group:FaceDetail opt
"""

# ==========================================
# 2.在此貼上你的 Prompt 內容 (CONTENT)
# 格式: 
# 標題 (必須對應 Config 的第一欄)
# Label | Value
# ==========================================
RAW_CONTENT = """
畫風
  韓系水光 CG | Stylized K-beauty CG, glass skin render, creamy porcelain, glossy highlight, pastel lavender, studio lighting.
  日系柔霧數位 | Soft Japanese digital art, misty skin render, hydrated dewy glow, warm sakura tone, natural window light.
  遊戲高階 CG | High-end game CG, specular skin reflection, subsurface scattering, unreal engine 5 glow, detailed 3D render.
  新海誠動畫 | Makoto Shinkai anime style, halo glow skin, saturated blue-purple tone, soft lens flare, cinematic anime art.
  寫實油畫感 | Hyperrealistic oil painting style, visible brush strokes, oily skin texture, Rembrandt lighting, amber tones.
  WLOP 數位 | WLOP style digital painting, pearlescent skin, iridescent highlight, soft blended glow, elegant ethereal art.
  UE5 物理渲染 | UE5 photorealism render, lumen skin reflection, metallic glossy texture, raytracing, professional 3D masterpiece.
  奢華緞面 CG | Cinematic CG render, luxury satin skin, warm candle light, teal and orange rim light, high-end hotel vibe.
  果凍通透 3D | Translucent jelly skin 3D render, strong SSS, pinkish undertone, soft diffused backlight, dreamy pastel art.
  黃金暮色藝術 | Stylized 3D art, honey-glazed skin render, golden hour backlight, dramatic shadows, warm bronze atmosphere.
  賽博液態渲染 | Cyber liquid gloss render, neon reflections, metallic lustre, cyan and magenta contrast, futuristic tech sheen.
  法式繪畫感 | Vintage art style render, soft focus, ivory skin tone, natural golden glow, delicate film grain, poetic mood.
  極致白瓷  | K-pop idol CG, extreme porcelain white skin, high-contrast glossy highlight, dazzling stage lights, flawless render.
  晨曦透亮  | Ethereal morning glow, dewy skin render, sunlight through sheer curtains, airy atmosphere, delicate Japanese art.
  幻想魔法  | Magical anime render, glowing skin, floating sparkles, vibrant purple-gold tone, dynamic fantasy lighting art.
  棉糖柔光  | Cute 3D render, marshmallow skin texture, soft ambient occlusion, pastel candy tones, lovable stylized glow.
  V-Ray 專業渲染 | Professional V-Ray render, flawless skin, soft global illumination, high-end 3D finish, masterpiece digital art.
  懷舊高畫質動畫 | Retro 90s anime aesthetic, modern high-fidelity render, grainy glow skin, neon city lights, nostalgic vibe.
  珍珠水潤數位 | Moist pearlescent skin render, liquid highlight, soft blended digital paint, elegant shimmering iridescent finish.
  星夜夢幻渲染 | Dreamy star-lit 3D art, ethereal glowing skin, soft bokeh, cool blue moonlight, magical light particles render.

體型
  仙系細腰控 | average-height delicate slim grace, softly tapering narrow waist with gentle lines, small perky breasts subtle firmness and quiet poise
  飄飄仙女線 | moderate stature ethereal lean elegance, slim hips with natural flow, modest high-sitting breasts light lift and barely shifting
  文青清瘦款 | balanced height refined slender form, graceful narrow waist soft curves, delicate round breasts elegant poise minimal movement
  安靜偷心型 | average-height graceful slim silhouette, defined waist gentle taper, small elegant breasts subtle natural lift
  精瘦腹肌控 | average-height athletic toned slimness, crisp narrow waist defined lines, small perky breasts firm lift with light subtle bounce
  腿長九頭身 | moderate stature sharp lean elegance, slim hips streamlined taper, modest pointed breasts high poised position
  帥感紙片人 | balanced height sleek slender form, graceful tapering waist, delicate round breasts natural shift and poised firmness
  走路會發光 | average-height streamlined slim grace, defined waist crisp flow, small elegant breasts light jiggle on gentle movement
  軟綿抱爆款 | average-height soft hourglass warmth, gently cinched waist plush hips, full round breasts gentle overflow and soft sway
  邪惡S曲線 | moderate stature classic voluptuous allure, slim waist thick soft thighs, heavy teardrop breasts hypnotic gentle movement
  多汁壞壞型 | balanced height juicy feminine charm, narrow waist jiggly lower body, perfectly round breasts natural bounce and warmth
  溫柔肉桂捲 | average-height sensual plush curves, cinched waist wide swaying hips, large heart-shaped breasts deep subtle cleavage
  貴婦肉感派 | moderate stature luxurious soft dream, slim waist thick creamy thighs, generous firm breasts gentle lift and soft jiggle
  曲線核彈級 | balanced height bombshell overflowing softness, narrow waist plush hips, overflowing round breasts liquid-like gentle sway
  壞壞小隻馬 | extra-petite delicate slim grace, softly narrow waist gentle taper, small perky breasts light firmness and subtle poise
  口袋小妖精 | tiny ethereal lean form, slim hips soft lines, modest high-sitting breasts quiet elegance barely shifting
  小隻緊實款 | extra-petite athletic toned slimness, crisp narrow waist defined lines, small perky breasts firm lift subtle bounce
  壞壞娃娃裝 | tiny sharp slender poise, defined waist crisp taper, delicate pointed breasts high poised position
  小隻肉肉球 | extra-petite soft hourglass warmth, gently cinched waist plush hips, full round breasts gentle overflow soft sway
  口袋Oversize | tiny classic voluptuous allure, slim waist thick soft thighs, heavy teardrop breasts hypnotic movement
  小隻多汁包 | petite juicy feminine charm, narrow waist jiggly lower body, perfectly round breasts natural bounce
  娃娃偷心款 | doll-like sensual plush softness, cinched waist wide swaying hips, large heart-shaped breasts subtle deep cleavage
  高挑仙界感 | towering delicate slender grace, endless legs soft taper narrow waist, small perky breasts subtle firmness quiet poise
  仙界腿精版 | statuesque ethereal lean lines, super long legs slim hips, modest high-sitting breasts light lift barely shifting
  高瘦高冷感 | amazonian refined slim poise, ultra-long legs graceful narrow waist, delicate round breasts subtle natural flow
  長腿雲霧感 | leggy graceful slim silhouette, endless legs defined waist gentle curves, small elegant breasts minimal movement
  高挑精實款 | towering athletic toned slimness, super long legs crisp narrow waist, small perky breasts firm lift light bounce
  伸展台模感 | statuesque sharp lean elegance, ultra-long legs streamlined taper, modest pointed breasts high poised firmness
  長直帥氣款 | amazonian sleek slender form, endless legs tapering waist crisp lines, delicate round breasts subtle shift
  高瘦流線型 | leggy streamlined slim grace, super long legs defined flow, small elegant breasts light jiggle graceful movement
  氣場爆棚型 | towering poised lean allure, ultra-long legs narrow waist sharp definition, modest round breasts natural high position
  高挑軟綿綿 | towering soft hourglass warmth, endless legs gently cinched waist plush hips, full round breasts gentle overflow sway
  長腿S女王 | statuesque classic voluptuous allure, super long legs slim waist thick soft thighs, heavy teardrop breasts hypnotic movement
  多汁女神款 | amazonian juicy feminine charm, ultra-long legs narrow waist jiggly lower body, perfectly round breasts natural bounce
  肉肉天仙款 | towering sensual plush softness, endless legs cinched waist wide swaying hips, large heart-shaped breasts deep cleavage
  曲線海嘯級 | leggy luxurious soft dream, super long legs slim waist thick creamy thighs, generous firm breasts soft lift jiggle
  長腿炸彈款 | statuesque bombshell overflowing curves, ultra-long legs narrow waist plush hips, overflowing round breasts liquid sway
  大地女神款 | amazonian fertility goddess grace, endless legs cinched waist wide breeding hips, massive round breasts natural lift warmth
  極限肉肉神 | towering maximum sensual softness, super long legs slim waist thick jiggly lower body, enormous teardrop breasts gentle hypnotic jiggle
  夢幻山景包 | leggy seductive plush dream, ultra-long legs narrow waist thick creamy hips, giant hemisphere breasts subtle liquid movement
  高挑完美曲 | statuesque hourglass perfection, endless legs cinched waist plush overflow, massive firm breasts light bounce on top
  長腿軟肉感 | amazonian ultimate soft allure, super long legs narrow waist thick velvet thighs, enormous round breasts gentle high jiggle
  慵懶犯罪線 | towering hypnotic voluptuous form, ultra-long legs slim waist overflowing lower curves, generous overflowing breasts soft natural sway
  雲端棉花山 | leggy heavenly plush curves, endless legs cinched waist thick sensual hips, large firm breasts subtle warmth and lift

臉型
  韓系銳利知性臉 | korean fashion sharp intellectual face, high cheekbones, piercing almond eyes
  韓系無表情精緻臉 | korean expressionless perfect face, symmetrical features, glossy lips
  韓系清新優雅臉 | korean fashion fresh elegant face, soft yet defined features, gentle sophisticated aura
  韓系清純偶像臉 | korean pure idol face, innocent wide eyes, minimal makeup, milky skin
  韓系清冷學姐臉 | korean cold senior sister face, sharp cat eyes, minimal expression
  韓系清冷貴氣臉 | korean icy aristocratic face, flawless symmetry, subtle contemptuous gaze
  韓系高冷財閥千金臉 | korean chaebol heiress face, expensive features, slight smirk
  韓系玻璃娃娃臉 | korean glass doll face, reflective glossy skin, perfect idol proportions
  韓系冷豔模特臉 | korean high fashion model face, strong bone structure, sultry half-lidded eyes
  韓系冷感高級臉 | korean cold apathetic beauty face, razor-sharp features, icy elegant expression
  韓系冷白皮貴族臉 | korean snow-white aristocratic face, pale flawless skin, haughty expression
  韓系冰山美人臉 | korean ice queen face, sharp jawline, emotionless perfect features
  精靈混血尖耳臉 | half-elf delicate face, slightly pointed ears, crystalline eyes
  精靈般纖細仙氣臉 | elf-like delicate ethereal face, sharp chin, luminous huge eyes, translucent skin
  精靈公主高貴臉 | elven princess royal face, regal bone structure, luminous skin
  夢幻發光精靈臉 | luminous elven fairy face, glowing skin, sparkling oversized eyes
  童話公主夢幻臉 | fairy-tale dreamy princess face, soft focus, magical innocence
  瓷娃娃無瑕臉 | unrealistically smooth porcelain doll face, zero pores, perfect symmetry
  仙氣混血臉 | ethereal mixed-race face, sharp yet soft features, glowing translucent skin
  仙女級發光肌臉 | fairy glowing skin face, soft radiant aura, dreamy soft focus eyes
  日系櫻桃小嘴臉 | japanese cherry lips face, small pouty mouth, cute button nose
  日系櫻花少女臉 | japanese sakura girl face, soft pink cheeks, cherry blossom aura
  日系鄰家元氣臉 | japanese girl-next-door lively face, natural blush, bright innocent smile
  日系軟糯治癒臉 | japanese soft healing face, gentle smile, warm innocent vibe
  日系軟糖甜妹臉 | japanese candy sweet girl face, pouty lips, glossy hydrated look
  日系軟萌圓臉 | japanese soft round baby face, chubby cheeks, sparkling doe eyes
  日系清新文藝臉 | japanese fresh literary girl face, gentle smile lines, natural wind-blown hair
  日系青春娃娃臉 | japanese youthful cute doll face, big round eyes, button nose, porcelain skin
  日系永遠18歲臉 | japanese forever 18 face, youthful eternal glow, big bright eyes
  日系水潤清純臉 | japanese dewy youthful face, glassy hydrated skin, soft natural makeup
  日系水汪汪大眼臉 | japanese watery big eyes face, teary innocent look, long lashes
  天女柔光臉 | celestial maiden soft face, gentle halo glow, serene otherworldly beauty

特別場景
  跨坐在三角木馬上，木棱卡在大腿根最深處 | triangle wooden horse between thighs, sharp edge pressing deeply into crotch
  跪趴在圓木馬上，木柱頂在大腿根 | round sybian-style barrel under pelvis, pressing upward between thighs
  背靠著高木馬，雙腿張開跨在兩側 | leaning back against tall wooden horse, legs spread wide over both sides
  雙手抱鋼管，雙腿纏繞倒掛 | spinning upside down on pole, legs wrapped tightly around metal pole
  背靠鋼管滑下，大腿內側緊貼冰冷金屬 | back against pole sliding down slowly, inner thighs pressed against cold steel
  單腿勾鋼管高抬，另一腿伸直 | single leg hooked high on pole, other leg extended straight
  跪地抱鋼管，胸口貼管上下磨蹭 | kneeling hugging pole, chest sliding up and down against metal
  坐在八爪椅中央，雙腿被皮帶固定張開到極限 | seated in center of octopus chair, legs strapped wide open to maximum spread
  仰躺八爪椅，膝蓋被扣環拉到肩膀高度 | lying back in octopus chair, knees pulled up to shoulder level by restraints
  側躺在八爪椅，單腿被吊高固定 | side-lying on octopus chair, one leg suspended high by strap
  雙手握吊環懸空，雙腿M字大開 | hanging from aerial rings, legs in wide M-shape
  倒吊在吊環上，長髮垂地腿部張開 | upside down on rings, legs spread with hair hanging to floor
  單手握吊環旋轉，雙腿纏繞環圈 | spinning on single ring, legs wrapped around metal loop
  背靠聖安德魯十字架，手腳被綁四角 | back against St. Andrew's cross, wrists and ankles bound to corners
  面朝X架趴上去，臀部高翹 | X-cross face down, hips raised high against wooden beams
  跪在細鐵籠裡，膝蓋頂著欄杆 | kneeling inside narrow metal cage, knees pressing against bars
  站立小鐵籠，雙手從欄杆外伸出被綁 | standing in tiny cage, arms extended through bars and bound outside
  坐在情趣鞦韆上，腿掛在兩側鏈條 | sex swing seat, legs draped over side chains
  仰躺情趣鞦韆，背部懸空腿被吊高 | lying back in sex swing, back suspended, legs pulled high
  跪在鞦韆座上，臀部懸空 | knelt on swing seat, hips hanging off the edge
  仰躺情趣吊床，雙腿被繩吊到天花板 | lying in bondage hammock, legs hoisted upward by ropes
  側躺在網狀吊床，腿從網洞穿出 | side-lying in web hammock, legs protruding through net holes
  跪在懺悔跪台上，膝蓋陷入軟墊 | kneeling on padded prie-dieu, knees sinking deep into cushion
  趴在高跪台上，臀部高過頭部 | face down on tall kneeling bench, hips higher than head
  雙手被吊在頂架，腳尖勉強點地 | hands suspended from overhead frame, toes barely touching floor
  倒吊在吊架，長髮掃地腿部大開 | upside down suspension, hair sweeping floor, legs spread wide
  站立在慢速旋轉台上，雙手背後綁 | standing on slowly rotating platform, wrists bound behind back
  跪在旋轉圓台上，膝蓋分開固定 | kneeling on spinning disc, knees strapped apart
  背靠牆壁，手腕被牆上鐵環拉高固定 | back against wall, wrists pulled high by wall rings
  面朝牆壁站立，腳踝被低環固定分開 | facing wall, ankles spread and locked to low wall rings
  跪在落地鏡前，大腿貼鏡面磨蹭 | kneeling in front of full-length mirror, thighs pressed against glass
  背靠鏡子滑下，雙腿大開反射在鏡中 | back sliding down mirror wall, legs wide open reflected clearly
  趴在情趣按摩床上，中間開洞臀部剛好卡在洞口 | face down on massage table with glory hole, hips positioned directly over opening
  仰躺按摩床，腿被兩側固定架拉開 | lying on back on bondage massage table, legs pulled wide by side restraints
  坐在束縛椅上，雙腿被椅腳固定大開 | seated in bondage chair, legs strapped wide to chair legs
  背靠束縛椅傾斜，膝蓋被拉到胸前 | reclined in bondage chair, knees pulled to chest by straps
  躺在真空床內，身體輪廓緊貼透明膠膜 | inside vacuum bed, body contours tightly pressed against clear latex sheet
  側躺在真空床，呼吸管從嘴伸出 | face-up in vacuum bed, breathing tube visible from mouth
  跪在懸吊鐵籠裡，膝蓋頂籠底 | kneeling inside hanging metal cage, knees pressing against floor bars
  站立吊籠，手從頂部欄杆伸出被綁 | standing in suspended cage, arms extended upward through top bars
  坐在中世紀拷問椅，腿被木枷固定 | spread on medieval interrogation chair, legs locked in wooden stocks
  仰躺在牙醫式問詢椅，腿架高分開 | reclined in gynecological exam chair, legs in stirrups wide apart
  坐在馬桶形束縛椅，臀下完全懸空 | seated on toilet bondage chair, hips completely exposed underneath
  跪趴馬桶椅，膝蓋固定在兩側 | kneeling over toilet chair, knees strapped to sides
  仰躺可旋轉圓盤台上，腿被固定在邊緣 | lying on spinning bondage wheel, legs strapped to outer rim
  站立綁在直立轉盤，緩慢旋轉 | standing bound to upright spinning wheel, rotating slowly
  站在開啟的鐵處女前，背部貼滿釘刺背板 | standing inside open iron maiden, back pressed against spiked panel
  仰躺拉伸架，四肢被繩索拉到四角 | lying on rack, limbs stretched to four corners by ropes
  趴在拉伸架，臀部被墊高四肢拉開 | face down on rack, hips raised by bolster, limbs pulled taut
  跪在旋轉展示台上，膝蓋分開固定 | kneeling on rotating display pedestal, knees strapped wide to base
  站在齊胸深的水箱裡，水面晃動拍打胸口 | standing in chest-deep water tank, water splashing against chest
  跪在水箱底，手腕被鎖在水面以上鐵環 | kneeling on tank floor, wrists locked to rings above waterline
  坐在老式電椅，雙腿被厚皮帶固定在椅腳 | strapped into vintage electric chair, legs spread and belted to chair legs
  仰躺在現代電療椅，膝蓋被金屬扣環拉開 | reclined in medical electric chair, knees locked wide by metal stirrups
  踮腳站在絞刑架活板門上，脖子套繩但未收緊 | standing on tiptoes on gallows trapdoor, noose loose around neck
  跪在絞刑台邊，手腕反綁在背後橫木 | kneeling beside gallows, wrists tied behind horizontal beam
  雙手被銬在高低兩根鐵柱之間，身體被迫前傾 | hands cuffed between high and low pillory posts, body forced to bend
  單腿抬高銬在高柱，另一腳勉強踮地 | one leg raised and cuffed to tall post, other toe barely touching ground
  坐在西班牙蜘蛛椅，背後尖刺輕貼皮膚 | seated in Spanish spider chair, spikes lightly pressing into back
  仰躺蜘蛛椅，金屬爪臂懸在胸前上方 | lying back under spider arms, metal claws hovering inches above chest
  跪趴在鴨凳上，膝蓋與手肘陷入凹槽 | kneeling on ducking stool, knees and elbows sinking into grooves
  被綁在可傾斜鴨凳，頭部朝下懸空 | strapped to tilting ducking stool, head hanging downward
  戴著重型鐵面具，只能從小孔呼吸，跪姿 | iron mask locked on head, breathing through small holes while kneeling
  鐵面具＋頸枷組合，迫使低頭 | iron mask combined with heavy neck yoke, forcing head down
  頸部與雙手鎖在木製頸手枷，彎腰站立 | neck and wrists locked in wooden pillory, bent over standing
  頸手枷＋腳枷全套，呈90度彎腰 | neck, wrists and ankles all in stocks, body folded at 90 degrees
  站在現代透明鐵處女內，門半開，內壁貼身 | inside clear modern iron maiden, door half-closed, spikes inches from skin
  仰躺穿刺桌，四肢被拉到桌角金屬環 | lying on impalement table, limbs stretched to corner rings
  跪在低矮穿刺桌，胸口貼桌沿 | chest pressed against edge of low impalement table while kneeling
  整個人塞進球形吊刑籠，只能蜷縮 | curled inside spherical hanging cage, barely fitting
  倒吊在長方吊籠，腳踝鎖頂部 | ankles locked to top of rectangular hanging cage, upside down
  跪在緩慢旋轉的圓柱鐵籠內 | kneeling inside slowly spinning cylindrical cage
  站立旋轉籠，手腕銬在籠頂 | standing in spinning cage, wrists cuffed to ceiling bars
  跪在窄水缸內，水淹到下巴 | kneeling in narrow water cylinder, water up to chin
  仰躺長條水槽，頭部剛好露出水面 | lying in long water trough, face just above surface
  背靠高燭台柱，手腕拉高綁在頂部 | back against tall candelabra pillar, wrists tied overhead
  跪地抱燭台柱，胸口緊貼冰冷大理石 | kneeling hugging marble candelabra pillar, chest pressed tight
  雙手高舉綁在審訊柱頂，腳踝分開綁底部 | wrists tied high on interrogation post, ankles spread and tied low
  背貼審訊柱滑下，膝蓋被迫張開 | back sliding down post, knees forced apart by base restraints
  雙臂反綁在粗羅馬柱後，胸部前挺 | arms tied behind thick Roman column, chest thrust forward
  單腿纏繞羅馬柱高抬，另一腿踮地 | one leg wrapped high around column, other on tiptoes
  倒掛在垂直鐵梯，腳踝鎖最高橫槓 | upside down on vertical metal ladder, ankles locked to top rung
  跪在鐵梯中段，膝蓋卡在橫槓間 | kneeling on ladder, knees trapped between rungs
  趴在窄吊橋板，橋身晃動 | face down on swaying suspension bridge plank
  跪坐吊橋中間，腿垂在兩側 | kneeling in middle of rope bridge, legs dangling off sides
  雙手握粗鐘繩被拉高，腳離地幾公分 | hands gripping thick bell rope pulled overhead, toes inches off floor
  跪地抱鐘繩，繩子從胯下穿過 | kneeling hugging bell rope running between thighs
  跨坐在尖頂刑凳，重量全壓在會陰區域 | straddling pointed judas cradle, full weight on crotch
  跪在低刑凳，尖頂頂在大腿根 | kneeling over low judas cradle, point pressing between thighs
  坐在鐵梨形刑具上，梨尖朝上 | seated on pear of anguish chair, metal pear pointing upward
  仰躺梨形台，梨尖對準下腹 | lying back over pear-shaped device, point aimed at lower abdomen
  身體半嵌進牆壁凹槽，四肢被牆環拉平 | body recessed into wall niche, limbs stretched flat by wall rings
  側身卡在牆壁狹窄開口，只能微動 | sideways trapped in narrow wall slot, minimal movement possible
  背靠火刑柱，手腕高綁柱頂 | back against stake, wrists tied high overhead
  跪地抱火刑柱，膝蓋分開卡在底座 | kneeling hugging stake, knees spread by base wedges
  單鉤從背後吊起整個人，腳尖點地 | single hook under shoulder blades lifting body, toes barely touching
  跪姿，背部吊鉤拉高迫使後仰 | kneeling with back hook pulling shoulders upward and back
  綁在可旋轉木十字架，緩慢轉動 | bound to slowly rotating wooden cross
  倒綁十字架，頭下腳上旋轉 | upside down on spinning St. Andrew's cross
  仰躺打開的石棺內，棺蓋半合 | lying inside half-closed stone sarcophagus
  跪在石棺內，棺蓋壓到肩膀高度 | kneeling inside sarcophagus, lid lowered to shoulder level
  整個人塞進大鐵桶，只露出頭和膝蓋 | crammed inside large metal barrel, only head and knees visible
  倒立鐵桶內，腳踝鎖桶口 | upside down inside barrel, ankles locked to rim
  跪在可逐漸縮小的鐵籠，籠壁慢慢逼近 | kneeling in gradually shrinking metal cage
  站立壓縮籠，手臂被迫貼身 | standing in compression cage, arms forced against sides
  跪在無數鏡子包圍的柱子前 | reflection of bound body on pillar surrounded by infinite mirrors
  背靠鏡面柱滑下，雙腿映出無限影像 | back sliding down mirrored pillar, legs reflected infinitely
  雙手被絞盤繩拉高，腳踝綁地錨 | wrists pulled high by winch, ankles anchored to floor
  仰躺絞盤台，四肢被四個絞盤拉開 | lying on winch table, limbs stretched by four separate winches
  跪在半埋沙箱，沙到胸口 | kneeling in sand box buried up to chest
  仰躺沙箱，沙慢慢覆蓋到脖子 | lying in sand box, sand gradually covering up to neck
  坐在冰冷尖釘鐵椅，重量全壓尖釘 | seated on cold spiked iron chair, full weight on spikes
  背靠尖釘椅背，雙手反綁椅側 | back against spiked chair back, wrists tied to sides
  四肢綁在水晶吊燈框架，隨燈緩慢搖晃 | limbs tied to chandelier frame, swaying gently
  倒掛吊燈底座，頭髮垂向地面 | upside down from chandelier base, hair hanging down
  跪在馬槽內，頭從槽邊伸出 | kneeling inside feeding trough, head protruding over edge
  趴在馬槽，臀部卡在槽口 | hips wedged in narrow feeding trough opening
  跪在巨型風箱拉桿下，隨拉動上下移動 | kneeling under massive bellows handle, moving up and down with each pull
  仰躺風箱板上，板隨鼓風起伏 | lying on bellows board, rising and falling with each pump
  跪在大型古董絞肉機前，膝蓋頂進料口 | kneeling before massive antique meat grinder, knees pressed to feed chute
  仰躺絞肉機台面，腿伸進機器下方 | lying under grinder table, legs extended beneath mechanism
  雙手綁水車葉片，被緩慢帶起又放下 | hands tied to waterwheel paddle, slowly lifted and lowered
  跪在水車軸心，隨轉動旋轉 | kneeling at waterwheel hub, rotating slowly
  跪在巨型竹蒸籠內，蒸汽上升 | kneeling inside massive bamboo steamer, steam rising
  仰躺多層蒸籠中間一層，蓋子半開 | lying in middle layer of stacked steamers, lid half open
  站在LED鐵處女內，內壁燈光緩慢閃爍 | inside illuminated modern iron maiden, inner spikes glowing
  跪在透明真空筒內，空氣被抽走身體貼壁 | kneeling inside clear vacuum cylinder, body pressed against glass as air is pumped out
  仰躺長真空筒，腳踝鎖在筒口 | lying in long vacuum tube, ankles sealed at opening
  仰躺拋石機投擲臂，隨角度變化傾斜 | lying on trebuchet throwing arm, tilting with counterweight
  跪在配重箱內，隨機器晃動 | kneeling inside counterweight box, swaying with mechanism
  頭和手伸出鴉籠，跪姿無法起身 | head and hands through scavenger's daughter cage, forced to kneel
  側躺鐵鴉籠內，身體被壓縮 | compressed sideways inside scavenger's daughter
  跪在巨型銅鐘內，隨敲擊震動 | kneeling inside massive bronze bell, vibrating with each strike
  倒掛鐘內，腳踝鎖鐘頂 | upside down inside hanging bell, ankles locked to clapper
  跪在古法橄欖壓榨機石板下 | kneeling under ancient olive press stone
  仰躺壓榨台，石輪緩慢滾過身體上方 | lying under rolling press stone, inches from body
  坐在絞刑掉落椅，腳下活板門半開 | seated on drop chair, trapdoor slightly ajar beneath
  跪在絞首椅前，脖子套繩但未拉 | kneeling before gallows chair, loose noose around neck
  蜷縮在小型鐵船內，隨水面搖晃 | curled inside small metal boat, rocking with water
  仰躺鐵船底，船身緩慢傾斜 | lying on bottom of tilting metal boat
  綁在投石機彈弓皮帶，隨拉伸後仰 | bound to catapult sling, pulled back with tension
  跪在投石機基座，隨機臂晃動 | kneeling on catapult base, swaying with arm
  仰躺在老式牙科椅，腿架高分開固定，頭燈直射臉 | reclined in vintage dental chair, legs in high stirrups, overhead lamp blinding face
  側躺牙科椅，膝蓋被金屬托架強制彎曲 | side in dental chair, knees forced into bent position by metal supports
  仰躺電動婦科檢查台，腿被自動分腿器緩慢撐開 | lying on powered gyno table, legs slowly spread by motorized spreader bars
  跪趴婦科台，臀部卡在圓形開口正上方 | kneeling face-down on gyno table, hips directly over circular opening
  四肢綁冰冷不鏽鋼手術台，燈光從正上方打下 | limbs strapped to cold stainless operating table, surgical lights directly overhead
  側躺手術台，身體被真空墊吸住固定 | side-lying on OR table, body sucked immobile by vacuum mattress
  躺在精神科束縛床，厚皮帶橫跨胸腰腿 | secured on psychiatric restraint bed with wide leather straps across chest waist thighs
  仰躺舊式四點束縛床，四肢拉到極限 | four-point restraint on antique psych bed, limbs stretched to maximum
  仰躺移動式X光台，鉛板壓在下腹固定 | lying on portable X-ray table, heavy lead apron pressing down on lower abdomen
  跪在X光台下，胸部貼影像板 | kneeling under X-ray plate, chest pressed flat against cassette
  仰躺頸椎牽引床，頭被金屬架固定拉長 | lying on cervical traction bed, head locked and slowly stretched
  趴在腰椎牽引台，骨盆被皮帶往反方向拉 | face down on lumbar traction table, pelvis pulled opposite direction by harness
  坐在1930年代電療椅，電極線垂在大腿內側 | seated in vintage electroshock chair, electrode wires dangling between thighs
  仰躺現代TMS磁療椅，線圈緊貼恥骨上方 | lying under transcranial magnetic stimulation chair, coil pressed just above pubic bone
  仰躺慢速傳送帶，雙手被磁鐵吸住上方鐵軌 | lying on slow conveyor belt, wrists magnetically pinned to overhead rail
  跪在傳送帶上，膝蓋被側邊夾具固定 | kneeling on moving belt, knees locked by side clamps
  站進工業熱縮機，熱風從四面吹來身體緊縮 | standing inside industrial heat shrink tunnel, hot air blasting from all sides
  仰躺液壓壓縮平台，平板緩慢下降壓住身體 | lying under hydraulic compression platen slowly descending
  跪在舊鈷60輻射室中央，厚重鉛門半關 | kneeling in center of vintage cobalt-60 irradiation chamber, lead door half closed
  背靠輻射室牆，雙手被磁扣吸在牆上 | back against irradiation chamber wall, wrists magnetically locked
  跪在大型注塑機模具前，膝蓋卡進模具凹槽 | kneeling before massive injection mold, knees trapped in mold cavity
  仰躺注塑機下，模具緩慢合攏停在身體上方 | mold halves slowly closing inches above body on injection machine bed
  仰躺真空成型台，熱塑板被吸下貼身 | lying on vacuum forming table, hot plastic sheet sucked down over body
  跪姿真空成型，塑膠膜緊貼胸部輪廓 | kneeling while plastic film vacuum-sealed around torso
  跪在古老懺悔室鐵格柵前，膝蓋頂著木板 | kneeling at confessional grille, knees pressing hard wooden kneeler
  背靠懺悔室牆，雙手從牆洞伸出被綁 | back against confessional wall, arms extended arms bound through glory holes
  仰躺大理石祭壇，四肢綁向四角十字 | lying on marble altar, limbs stretched to four corner rings
  跪在祭壇台階，胸口貼冰冷大理石階梯 | kneeling on altar steps, chest pressed to cold marble
  背靠巨大木十字架，手腕綁橫木無法垂下 | back against massive wooden cross, wrists tied high on crossbeam
  跪抱十字架底部，膝蓋分開卡在底座 | kneeling hugging base of cross, knees forced apart by pedestal
  站炎柱中央，熱氣從下方鐵網上升 | standing on hell pillar grate, hot air rising from below
  跪在煉獄柱平台，鐵欄圍住腰部 | kneeling on narrow purgatory pillar platform surrounded by iron bars
  頸手鎖在中世紀修女枷，彎腰90度 | nun in wooden branks pillory, bent at 90 degrees
  戴口枷式頭罩跪地，鐵鏈連到地板 | hooded with scold's bridle, chain to floor while kneeling
  仰躺旋轉雕塑轉台，被黏土緩慢覆蓋 | lying on rotating pottery wheel while clay slowly applied over body
  跪在雕塑台上，藝術家手繞身體塑形 | kneeling on pedestal while invisible hands mold clay around torso
  四肢綁現代懸吊藝術裝置，緩慢旋轉360度 | suspended in kinetic art installation, rotating 360 degrees
  倒掛藝術吊架，頭髮掃地身體隨風擺動 | upside down in gallery suspension rig, swaying gently
  跪在美術館透明壓克力箱內，觀眾圍觀 | kneeling inside museum clear acrylic box, surrounded by viewers
  仰躺透明展箱，箱蓋緩慢下降壓縮空間 | lying in clear display case, lid slowly lowering
  站進全息光柵房間，光束從各角度掃過身體 | standing in holographic light installation, beams scanning body
  跪在雷射網格中央，紅點在皮膚上游走 | kneeling in laser grid, red dots crawling over skin
  仰躺工業滾筒乾燥機內，熱風循環 | inside massive industrial roller dryer, hot air circulating
  跪在滾筒內壁，隨轉動翻滾 | kneeling inside slowly rotating drying drum
  仰躺負壓隔離艙，透明罩蓋到胸口 | lying in negative pressure isolation pod, clear dome down to chest
  跪在小型隔離艙，臉貼透明艙壁 | kneeling in small isolation chamber, face pressed to clear wall
  坐軍事審訊椅，強光燈直射臉部雙腿分開 | seated in military interrogation chair under blinding lamp, legs spread
  仰躺水刑台，布蓋臉水緩慢傾倒 | lying on waterboarding table, cloth over face, water pouring slowly
  蜷縮進透明休眠膠囊，艙門緩慢關閉 | curled inside clear stasis pod, door slowly sealing
  仰躺科幻膠囊，玻璃蓋降下貼身 | lying in futuristic capsule, glass lid lowering to body contour
  跪在零下冷凍隧道中央，霧氣環繞 | kneeling in cryogenic freezing tunnel, frost forming
  仰躺傳送帶進入冷凍隧道，冰晶覆蓋 | lying on belt entering freezing tunnel, ice crystals forming
  跪抱苦路第9站石柱，膝蓋磨地 | kneeling hugging Station IX pillar, knees grinding stone floor
  背靠第3落石柱，雙手高舉綁橫木 | back against Station III pillar, arms raised and bound
  站進液態矽膠模具池，矽膠緩慢上升 | standing in liquid silicone mold pool, silicone rising to neck
  仰躺人體複製模具，矽膠灌入貼身 | lying in full body duplication mold, silicone filling
  跪進退役核電廠輻射訓練艙，警報燈閃 | kneeling inside decommissioned reactor training chamber, warning lights flashing
  仰躺體外震波碎石台，震波頭對準下腹 | lying on lithotripsy table, shockwave head aimed at lower abdomen
  側躺震波台，膝蓋被固定架頂開 | side-lying with knees forced apart by lithotripter frame
  四肢被氣動夾具固定在工作台，夾具緩慢收緊 | limbs held by pneumatic clamps on workbench slowly tightening
  跪在夾具台，膝蓋被液壓夾住 | kneeling with knees trapped in hydraulic vise
  仰躺離心重力訓練室，牆壁高速旋轉 | lying in spinning centrifuge room walls blurring
  跪在零重力模擬艙，頭髮漂浮起 | kneeling in zero-G trainer hair floating
  綁在慢慢轉動的贖罪輪，頭下腳上 | upside down on slowly rotating penance wheel
  跪在贖罪輪胎內側，隨轉動起伏 | kneeling inside giant penance wheel rising and falling
  跪進透明呼吸雕塑，胸口起伏讓雕塑膨脹 | kneeling inside breathing sculpture chest movement inflating structure
  蜷縮進工業蒸壓釜，蒸汽高壓噴射 | curled inside industrial autoclave high pressure steam jets
  仰躺透明高壓氧艙，艙壓緩慢上升 | lying in clear hyperbaric chamber pressure rising
  仰躺黑色感官剝奪水槽，完全漂浮 | lying in pitch black sensory deprivation tank floating
  跪在剝奪槽內，耳塞眼罩全黑 | kneeling in tank with total blackout and ear plugs
  仰躺劇烈振動測試台，身體隨頻率震動 | lying on violent vibration test platform shaking at different frequencies
  跪在振動台，膝蓋被吸盤吸住 | kneeling with knees vacuum-sucked to shaking platform
  坐進腦波控制椅，頭部線圈閃爍 | seated in neural control chair head coils pulsing
  仰躺腦機介面床，電極貼滿頭皮 | lying on brain-computer interface bed electrodes covering scalp
  綁在中世紀火刑輪上緩慢轉動 | bound to medieval breaking wheel slowly rotating
  跪在火刑輪胎內側，輪軸轉動 | kneeling inside catherine wheel as it turns
  跪進液態金屬藝術浴池，金屬到腰 | knelt in liquid metal art pool up to waist
  四肢綁材料拉伸試驗機夾頭，緩慢拉長 | limbs in tensile testing machine jaws slowly stretching
  綁在旋轉精神病治療床，床身轉動 | strapped to spinning psychiatric rotating bed
  站進時間扭曲艙，鏡面反射無限身體 | standing in time dilation chamber infinite mirror reflections
  跪在工業高溫爐觀察窗前，熱浪撲面 | kneeling before industrial furnace observation window heat blasting
  跪在零下冰浴醫療槽，冰水到胸 | knelt in sub-zero medical ice bath water to chest
  仰躺高空低壓模擬艙，氧氣面罩貼臉 | lying in altitude chamber oxygen mask sealed
  仰躺巨型沙漏下半部，沙緩慢流下身體 | lying in lower half of massive hourglass sand flowing over
  跪在沙漏中央，沙從四面八方壓迫 | kneeling in hourglass center sand pressing from all sides
  仰躺1000噸沖壓機下，模具停在距離身體1cm | under 1000-ton press mold frozen 1cm above body
  跪在沖壓機台面，膝蓋被模具邊緣卡住 | knees trapped by descending press edges
  跪在奈米霧室，霧氣沿身體爬行 | kneeling in nanobot mist chamber fog crawling over skin
  仰躺古老聖水洗禮槽，水滿溢到脖子 | lying in ancient baptismal font water overflowing to neck
  跪在聖水槽內，水位緩慢上升 | kneeling in font water slowly rising
  仰躺1930年代電擊治療台，電極貼太陽穴大腿 | lying on vintage ECT table electrodes on temples and thighs
  跪在熱蠟藝術裝置，蠟沿身體流下 | kneeling while hot wax installation pours over body
  跪在真空鍍膜艙中央，金屬蒸汽沉積 | kneeling in vacuum deposition chamber metal vapor coating skin
  跪在訓練用毒氣室，面罩外霧氣瀰漫 | kneeling in gas chamber training mask outside fog
  仰躺在反重力場發生器上，頭髮漂浮身體懸空 | lying in anti-gravity field generator hair floating body levitating
  背靠會呼吸的藝術牆，牆面隨呼吸起伏 | back against breathing wall surface expanding and contracting
  坐進高速離心機座艙，G力壓身 | seated in human centrifuge G-forces pressing body
  跪在垂直風洞，強風從下方托起 | kneeling in vertical wind tunnel upward blast
  仰躺填充碎冰的維多利亞冰棺 | lying in Victorian ice coffin filled with crushed ice
  四肢綁永動機曲柄，隨機器永遠轉動 | limbs tied to perpetual motion machine cranks turning forever
  仰躺海邊白色吊床，海浪聲在腳下 | lying in white beach hammock waves crashing below feet
  跪在吊床中央，網繩陷入大腿根 | kneeling in hammock center netting pressing into upper thighs
  仰躺漂浮在淺水區的巨型漂浮床 | lying on massive floating daybed in shallow turquoise water
  跪在漂浮床上，水面輕輕拍打膝蓋 | kneeling on floating bed water lapping at knees
  跪在懸空玻璃地板房間，海面就在腳下幾公分 | kneeling on glass floor overwater bungalow ocean inches below
  仰躺玻璃地板，魚群游過身下 | lying on clear glass floor tropical fish swimming beneath
  跪在夜間私人停機坪，城市燈火在腳下展開 | kneeling on rooftop helipad city lights sprawling below
  仰躺停機坪中央，風從四面吹來 | lying in center of helipad wind swirling from all sides
  仰躺純白鹽磚床上，鹽粒在皮膚上融化 | lying on pure white salt brick bed grains melting on skin
  跪在鹽屋地板，鹽霧緩緩上升 | kneeling on salt floor gentle salt mist rising
  仰躺滿溢玫瑰花瓣的黑色大理石浴缸 | lying in overflowing black marble tub completely covered in rose petals
  跪在浴缸中，花瓣堆到胸口 | kneeling in tub rose petals piled to chest
  跪在巨型香檳塔最底層，香檳順大腿流下 | kneeling under massive champagne tower bubbles cascading down thighs
  仰躺香檳塔正下方，金色液體淋下 | lying directly under champagne pyramid golden liquid pouring over
  跪在紅絲絨包廂欄杆前，俯瞰舞台 | kneeling at velvet opera box rail overlooking stage
  仰躺包廂沙發，帷幕緩慢拉開 | lying on opera box sofa heavy curtains slowly parting
  跪在全玻璃地板的100樓酒廊，雲層在腳下 | kneeling on glass floor of 100th-floor sky lounge clouds below
  仰躺雲端酒廊無邊際沙发，城市變成玩具 | lying on infinity sofa city looking like toys far below
  仰躺冰酒窖長凳，寒氣在皮膚凝結水珠 | lying on ice wine cellar bench frost forming on skin
  跪在冰酒窖地板，呼吸變成白霧 | kneeling on frozen floor breath turning to mist
  仰躺觀光潛艇透明底部，海底世界在身下漂過 | lying on glass-bottom private submarine ocean floor gliding beneath
  跪在潛艇圓窗前，魚群貼窗游過 | kneeling at submarine porthole fish swimming right against glass
  跪在熱氣球籃底，雲海在腳下翻滾 | kneeling in hot air balloon basket sea of clouds below
  仰躺籃底，風輕輕搖晃身體 | lying in basket floor gentle swaying in breeze
  跪在露天雪中溫泉，雪花落在肩頭 | kneeling in open-air onsen snowflakes landing on shoulders
  仰躺雪地溫泉，蒸汽與雪花交織 | lying in snow onsen steam and snow mixing
  仰躺瀑布下的白色大理石長椅，水流按摩背部 | lying on marble bench under private waterfall water massaging back
  跪在瀑布台階，水流順著大腿流下 | kneeling on waterfall steps water running down thighs
  坐在花園高空鞦韆，城市夜景在腳下搖晃 | sitting on rooftop flower swing city lights swaying below
  跪在鞦韈座墊，藤蔓纏繞腳踝 | kneeling on swing cushion vines loosely wrapping ankles
  仰躺巨型圓頂IMAX座位，銀幕包圍全身 | lying in private domed IMAX seat screen wrapping around body
  跪在圓頂中央，星空影片360度播放 | kneeling in dome center galaxy film playing all around
  仰躺奢華沙漠帳篷絲綢地毯，風沙輕拍帳壁 | lying on silk carpets in luxury desert tent wind against canvas
  跪在帳篷中央，燭火搖曳投下長影 | kneeling in tent center candlelight casting long shadows
  跪在懸浮於峽谷的透明泳池邊緣 | kneeling at edge of cantilevered infinity pool over canyon
  仰躺池中漂浮墊，雲霧在腳下流動 | lying on floating mattress clouds drifting below
  仰躺全透明極光觀測屋，極光在頭頂流動 | lying in glass igloo northern lights flowing overhead
  跪在玻璃屋中央，綠光沿身體滑過 | kneeling under aurora green light sliding over skin
  跪在停機坪已熄火的直升機座椅上，旋翼餘風吹 | kneeling in parked helicopter seat residual rotor wind blowing
  仰躺直升機後艙皮椅，機身輕微搖 | lying on leather bench in helicopter cabin gentle swaying
  跪在火山口邊黑色溫泉池，蒸汽上升 | kneeling in black volcanic hot spring steam rising
  仰躺火山溫泉岩台，熱氣包圍 | lying on volcanic rock ledge hot mist enveloping
  跪在100樓透明吧台下，霓虹在腳下流動 | kneeling under glass bar counter neon flowing below
  仰躺吧台旁的無邊際沙发，雲層在膝蓋 | kneeling on infinity sofa clouds at knee level
  跪在巨型壁爐前，火光映紅皮膚 | kneeling before massive fireplace firelight glowing on skin
  仰躺壁爐前羊毛地毯，火舌舔舌 | lying on wool rug flames dancing
  跪在漂浮於泳池的透明餐桌上，水面齊胸 | kneeling on floating transparent dining table water to chest
  仰躺漂浮餐桌，氣泡從下方上升 | lying on floating table bubbles rising from below
  仰躺可移動圓頂天文台躺椅，銀河旋轉 | lying on moving observatory chaise galaxy rotating above
  跪在天文台中央，星空投影落在皮膚上 | kneeling under private planetarium stars projected on skin
  跪在四面落地鏡中央，鏡中無限反射身體 | kneeling in mirrored room, infinite reflections from all sides
  仰躺巨大圓形水床，水波隨呼吸起伏 | lying on massive circular waterbed, waves rippling with every breath
  跪在水床上，水面晃動到大腿根 | kneeling on waterbed, water lapping at upper thighs
  趴在深紅絲絨長台，布料貼身起伏 | face down on deep red velvet platform fabric clinging and moving
  仰躺絲絨台，雙腿沿邊緣垂下 | lying back on velvet stage, legs draped over edge
  仰躺緩慢旋轉的圓形大床，絲被纏繞身體 | slowly rotating round bed, silk sheets twisted around body
  跪在旋轉床中央，床緣旋轉 | knelt in center of spinning round bed
  坐在透明壓克力吊籃椅，雙腿從下方垂下 | sitting in clear acrylic hanging egg chair, legs dangling below
  仰躺吊籃椅，椅背傾斜，膝蓋掛在邊緣 | reclined in hanging egg chair, knees hooked over rim
  坐在廢棄雲霄飛車最前排，安全桿緊壓大腿 | strapped into front seat of abandoned rollercoaster, lap bar pressing thighs
  跪在飛車座椅上，安全桿卡在腰後 | kneeling on coaster seat, lap bar locked behind waist
  坐在摩天輪透明艙底，艙體緩慢上升 | lying on floor of glass ferris wheel gondola as it rises
  跪在摩天輪艙中央，艙搖晃 | kneeling in center of swaying ferris wheel cabin
  跨坐在旋轉木馬最高那匹馬，木馬柱之間 | straddling tallest carousel horse between pole and saddle
  仰躺旋轉木馬地板，裙邊被離心力掀開 | lying on spinning carousel floor fabric lifted by centrifugal force
  仰躺巨大圓形按摩浴缸，水流從下方噴射 | lying in massive circular jacuzzi jets pulsing upward
  跪在浴缸中，水流集中在大腿內側 | knelt in jacuzzi concentrated jets between thighs
  仰躺開啟的三角鋼琴蓋上，琴弦在身下震動 | lying on open grand piano lid strings vibrating beneath body
  跪在鋼琴蓋上，手指撥弦身體隨之顫 | kneeling on piano lid body trembling with plucked strings
  背靠巨大空白銀幕站立，投影燈從後打光 | standing back against massive blank cinema screen spotlight from behind
  跪在銀幕前，強力風機從下方吹 | knelt in front of screen powerful fans blowing upward
  跪在緩慢旋轉的攝影轉台上 | knelt on slowly rotating studio turntable
  仰躺轉台，燈光從四面八方打來 | lying on turntable spotlights hitting from all directions
  跪在全透明觀光電梯地板，城市夜景在腳下 | kneeling on glass elevator floor city lights far below
  背靠玻璃電梯壁緩慢下降 | back against glass wall as elevator descends
  坐在旋轉餐廳最外圈桌下，桌面緩慢轉動 | under slowly rotating restaurant table at outer edge
  跪在旋轉餐廳地板，桌腳圍繞身體 | knelt on rotating restaurant floor table legs circling body
  仰躺遊艇白色軟墊甲板，海浪輕晃 | lying on yacht white cushioned deck gentle waves rocking
  跪在甲板欄杆邊，海風吹起頭髮 | kneeling at yacht rail sea breeze lifting hair
  仰躺平豪華私人飛機寬大皮椅，安全帶低扣腰 | lying flat on private jet wide leather seat belt low across hips
  跪在飛機走道地毯，機身輕微顛簸 | kneeling in jet aisle turbulence rocking body
  跪在無邊際泳池邊緣，水面齊胸 | kneeling at infinity pool edge water at chest level
  仰躺池邊軟墊，腿伸進水面 | lying on poolside cushion legs trailing in water
  仰躺巨大溫室中央花床上，花瓣覆蓋身體 | lying among flowers in massive greenhouse bed petals covering body
  跪在花床中，藤蔓自然纏繞手腕 | knelt in floral bed vines loosely wrapping wrists
  跪在透明冰磚宮殿地板，寒氣升騰 | kneeling on clear ice brick floor mist rising
  仰躺冰床上，冰面映出倒影 | lying on bed of ice perfect reflection beneath
  仰躺圓頂天文台移動躺椅，星空投影旋轉 | lying on moving observatory recliner starfield rotating overhead
  跪在星空艙中央，銀河投影在皮膚流動 | kneeling under galaxy projector light flowing over skin
  坐在豪華單人劇院座椅，座椅緩慢震動 | in single luxury cinema seat gentle vibration mode
  跪在地毯階梯式座位間，銀幕光打在身上 | kneeling between tiered cinema seats screen light washing over body
  仰躺戶外溫泉平滑石台，蒸汽包圍 | lying on smooth outdoor onsen stone steam enveloping
  跪在溫泉石台，水流從下方石縫湧出 | kneeling on stone hot spring water bubbling up between thighs
  跪在透明冰屋內，極光在天頂閃爍 | kneeling inside clear igloo aurora dancing overhead
  仰躺毛皮鋪的冰床，呼吸化成白霧 | lying on fur-covered ice bed breath turning to mist
  仰躺在水晶吊燈正下方，燈光折射在皮膚 | crystal chandelier directly overhead light fracturing over skin
  跪在吊燈下，燈光隨呼吸微微晃動 | kneeling under chandelier crystals swaying with breath
  跪在熱帶植物包圍的室內瀑布下 | kneeling under indoor waterfall surrounded by jungle plants
  仰躺瀑布下的平滑大理石台 | lying on marble under private waterfall
  
服裝設計
  束線帶編織 (扎肉) | Zip-tie construction suit, hundreds of plastic fasteners connected, sharp plastic ends, digging into soft flesh, DIY punk aesthetic.
  銀色大力膠帶 (粗糙) | Silver duct tape wrapping, crude wrinkled texture, industrial adhesive, compressing flesh tightly, raw sticky residue.
  別針拼接 (縫隙) | Safety pin dress, fabric scraps held together by metal pins, dangerous gaps between connections, metal piercing fabric.
  工業重鏈 (垂墜) | Heavy industrial chains, cold iron texture, weight pressing into shoulders and hips, sagging against gravity.
  鐵絲網纏繞 (尖刺) | Barbed wire wrapping, metal wire with sharp spikes, dangerous aesthetic, rust texture, caution.
  透明塑膠袋 (皺褶) | Clear plastic recycling fashion, crinkly texture, multiple layers of thin plastic, noisy light reflections.
  OK繃微型覆蓋 | Adhesive bandage micro-bikini, using only band-aids to cover essentials, cross shape, medical adhesive texture.
  注水雙層PVC | Water-filled double-layer PVC, heavy sloshing texture, water distorting body shape inside the suit, liquid movement.
  液態乳膠塗層 (剝落) | Dried liquid latex bodypaint, uneven dripping edges, glossy rubber coating directly on skin, peeling edges.
  史萊姆服 (滴落) | Viscous slime suit, thick semi-liquid material, constant dripping, holding shape loosely, translucent color.
  融化黃金/金屬 | Melting gold dripping, thick metallic liquid texture covering body parts, heavy gloss, hardening edges.
  冰霜/凍結裝 | Frozen ice dress, melting water dripping, condensation mist, frost texture on skin, cold atmosphere.
  半透乳膠 (醫用) | Translucent natural latex, milky semi-transparency, skin tone and details visible through rubber, cloudy texture.
  冷光線纏繞 (EL Wire) | EL wire wrapped around naked body, glowing neon cables, cyberpunk lighting, wires pressing into skin.
  生物發光有機體 | Bioluminescent organic skin-suit, living texture, pulsing light patterns, veins glowing under thin membrane.
  極細蛛絲 (黏性) | Spider silk bodysuit, extremely fine white threads, barely there, sticky texture, trapping the body.
  破碎鏡面馬賽克 | Mosaic mirror dress, sharp glass shards attached to mesh, dangerous reflections, fragmented light.
  超薄氨綸 (撐開) | Ultra-thin spandex leotard, material stretching to transparency over bust/hips, shiny synthetic sheen, tension lines.
  寬版乳膠帶 (木乃伊) | Wide latex ribbon wrapping, mummy style, gaps revealing skin between layers, alternating tightness.
  呼吸管真空衣 (全包) | Full-body vacuum suit with breathing tube, latex sucked into ears/navel, total enclosure, restricted movement.
  透明PVC洋裝 | Clear PVC dress, high-gloss plastic texture, fully visible body underneath, plastic clinging to skin, condensation fog.
  超薄雪紡 (透視) | Ultra-sheer chiffon, translucent and airy fabric, draping over curves, seeing skin texture through fabric.
  水晶網紗 (閃亮) | Crystal netting dress, sparkling gemstones on thin threads, gaps revealing skin, light refracting.
  彩繪錯覺 (極薄) | Bodypaint style fabric, matte finish, zero wrinkles, absolute anatomical definition, second skin.
  保鮮膜 (纏繞) | Cling film wrapping, thin wrinkly plastic texture, adhering strictly to skin, distorted reflections.
  視覺網紋 (摩爾紋) | Moiré pattern mesh, optical illusion texture, semi-transparent layers, vibrating visual effect.
  霧化噴塗 (漸層) | Atomized spray-paint effect, gradient mist colors blending into skin, ethereal semi-transparent.
  霜化玻璃質感 | Frosted glass texture fabric, semi-opaque blur, soft matte finish, diffused light.
  貼膚乳膠 (黑亮) | Tight latex bodysuit, high polish finish, reflective surface, muscle definition highlighted.
  濕身白T (透肉) | Wet white cotton t-shirt, soaked fabric becoming translucent, clinging to skin, water droplets.
  油膜感乳膠 (虹彩) | Oil-slick latex, iridescent finish shifting colors, liquid poured over body look.
  液態金屬 (水銀) | Liquid chrome fabric, mirror-like metallic reflection, distorting surroundings, seamless glossy wrap.
  珍珠光澤 | Pearlescent sheen fabric, soft milky white reflections, smooth silky texture.
  全息微亮片 | Holographic micro-glitter, rainbow spectrum reflections, sparkling intensely under light.
  樹脂滴流上衣 | Hardened resin drip top, looking like melting liquid frozen in time, sculptural plastic texture.
  鏡面反射連體 | Mirrored chrome suit, full-body reflective surface, high-polish, futuristic aesthetic.
  真空收縮 (窒息) | Vacuum-sealed bodysuit, plastic sucked tight, flattened curves, claustrophobic tightness.
  大露背 (脊椎) | Extreme backless dress, exposing entire back to lower spine, shoulder blades visible.
  側胸挖空 (Sideboob) | Deep sideboob cutout, revealing side curve of breast, dangerous cut.
  下胸短版 (Underboob) | High-cut crop top, revealing underboob, teasing lower bust curve.
  V形骨盆高衩 | High-leg bodysuit, deep V-cut front accentuating pelvic bones, elongating legs.
  髖部鏤空 | Hip bone cutouts, revealing waist curve and hip dip, skin peeking through.
  極小比基尼 | Micro bikini, tiny triangular fabric patches, barely covering essentials, string ties.
  全開背丁字 | Open back thong bodysuit, revealing buttocks and back, high cut legs, sleek front.
  掛脖側露洋裝 | Halter dress, draping fabric, deep side cuts revealing torso profile.
  膠帶時裝 (幾何) | Black fashion tape, geometric shapes covering essentials, skin compression, raw edgy look.
  單絲張力 (隱形) | Invisible monofilament harness, creating deep indentations on skin, floating aesthetic.
  多重織帶 (戰術) | Complex webbing straps, nylon straps wrapping body, geometric bondage pattern.
  繩帶壓痕 (肉感) | Tight rope straps, digging into soft skin, visible redness and indentations (squish).
  鎖子甲內衣 | Metal chainmail lingerie, cold metal links against warm skin, heavy drape, see-through.
  珍珠身鏈 | Pearl body chain, chains of pearls draping over naked body, jewelry as clothing.
  鯨骨束衣 (極細腰) | Whalebone corset, extreme waist cinching, rigid hourglass silhouette, visible boning.
  下托框架 (無杯) | Underwire frame harness, cupping bust without fabric, structural lines on skin.
  大腿繫帶 (勒肉) | Thigh garters, straps wrapping tightly around thighs, squeezing flesh slightly.
  撕裂繃帶 | Torn gauze bandages, mummy wrapping, loose threads, dirty weathered texture.
  破碎網布 (龐克) | Shredded mesh fabric, uneven holes, grunge aesthetic, layered look.
  戰損焦灼 | Burnt clothing, scorched edges, ash stains, holes revealing skin.
  將裂未裂 (緊繃) | Bursting seams, clothes too tight, gaps appearing between stitches, extreme tension.
  雷射切割 (格柵) | Laser-cut lattice leather, geometric holes revealing skin, sharp shadow patterns.
  羽毛邊緣 | Feather-trimmed edges, soft plumage framing exposed skin, fluffy texture.
  毛絨邊框 | Plush fur-lined edges, fuzzy borders against bare skin, soft tactile contrast.
  電鍍金屬塗層 | Electroplated gold coating, rigid yet flexible metallic shell on skin, flaking effect.
  螢光邊緣 (發光) | Fluorescent glowing edges, neon outlines on dark fabric, cyber light trace.
  磁石吸附 (拼接) | Magnetic panel clothing, gaps between panels, modular fit, high-tech seamless.
  編織藤蔓 (蕾絲) | Woven vine-pattern lace, organic threads revealing skin, botanical texture.
  雷射投影 (虛擬) | Laser-projected pattern, holographic light designs on bare skin, ephemeral texture.
  泡泡紗浮雕 | Bubble gauze embossed, puffed fabric texture, airy feel against skin.
  鉚釘點綴 | Rivet-studded cutouts, metallic studs framing skin, industrial hardware contrast.
  蠟封邊緣 | Wax-dripped edges, hardened irregular borders, artistic melted illusion.
服裝搭配  
  魚網連體衣 | Full body fishnet, diamond pattern mesh, seamless fit over skin.
  白T牛仔褲 (經典) | Tight white t-shirt and skinny jeans, simple casual look, shirt tucked in, accentuating curves, denim texture.
  Oversized 帽T | Oversized hoodie, wearing nothing underneath (no pants), baggy sleeves hiding hands, loose fit contrasting bare legs.
  露肩毛衣 (殺死處男) | Off-shoulder knitted sweater, virgin killer style, open back, soft wool texture, cozy yet revealing.
  細肩帶背心 + 熱褲 | Camisole tank top and denim dolphin shorts, summer vibe, skin exposure, casual home wear.
  碎花洋裝 (森系) | Floral sundress, lightweight fabric, spaghetti straps, summer breeze, feminine and soft silhouette.
  緊身包臀裙 | Bodycon mini dress, solid color, hugging every curve, evening party wear, hemline high above knees.
  吊帶褲 (單肩) | Denim overalls (dungarees), one strap unbuckled hanging down, bandeau top underneath, playful mechanic vibe.
  改良旗袍 | Short Cheongsam (Qipao), high side slits, silk fabric with dragon embroidery, mandarin collar, hugging curves.
  兔女郎 | Bunny girl suit, strapless satin bodysuit (leotard), bunny ears headband, collar and cuffs, fishnet tights.
  哥德蘿莉 (洋裝) | Gothic Lolita dress, black frills and lace, bell-shaped skirt, ribbons, mary jane shoes, dark doll aesthetic.
  皮衣皮裙 (機車) | Biker leather jacket and mini skirt, crop top underneath, zippers and studs, cool edgy style.
  和服浴衣 (短版) | Short Yukata, traditional japanese floral pattern, sash (obi) tied at waist, loose neckline, festive vibe.
  絲綢睡裙 | Silk slip dress, satin fabric, thin straps, lace trim at hem, bedroom setting, soft luster.
  蕾絲內衣套裝 | Matching lace lingerie set, bra and panties, intricate floral transparency, seductive and intimate.
  寬鬆男友襯衫 | Oversized white button-up shirt, wearing only the shirt, buttons partially undone, messy morning look.
  娃娃裝 (Babydoll) | Sheer babydoll lingerie, loose flowing fabric from bust down, cute and sexy, visible panties.
  高領無袖毛衣 | Sleeveless turtleneck sweater, ribbed knit texture, tight fit on chest, bare arms, elegant silhouette.
  天鵝絨運動套裝 | Pink velour tracksuit (Juicy style), cropped zip-up hoodie, low-rise sweatpants, rhinestone "Baby" text on back, soft velvet texture.
  蝴蝶亮片肚兜 | Sequin butterfly top, backless halter design, shimmering sequins, diamond shape covering chest, tie-back strings, 2000s pop star vibe.
  低腰褲 + 鯨魚尾 | Ultra low-rise denim jeans, exposing hip bones, visible thong straps (whale tail), crop baby tee, midriff bearing.
  金屬鏈甲上衣 | Chainmail halter top, silver metal mesh draping, cowl neck, disco aesthetic, clubwear, seeing skin through links.
  短版絨毛外套 | Fuzzy cropped cardigan, only top button fastened, pastel color, fluffy texture, revealing stomach, cute and sexy.
  迷你百褶 + 泡泡襪 | Micro plaid mini skirt and loose leg warmers, platform sneakers, chunky belt, Japanese Gyaru influence.
  無框墨鏡 + 唇環 | Rimless tinted sunglasses (yellow/pink), lip piercing, baguette bag under arm, glossy lips, cool attitude.
  牛仔拼接馬甲 | Patchwork denim corset top, frayed edges, different shades of blue jeans fabric, strapless, structured bodice.
  漁網內搭 + 大T | Fishnet long sleeve top under oversized graphic t-shirt, layered look, ripped black jeans, choker necklace, grunge aesthetic.
  破壞感針織衫 | Distressed knit sweater, loose threads, holes revealing skin/bra, off-shoulder, thumb holes, moody vibe.
  格紋法蘭絨 (腰綁) | Plaid flannel shirt tied around waist, crop top, denim shorts, black boots, casual grunge style.
  皮革項圈 + 鉚釘 | Spiked leather choker and wristbands, heavy metal hardware, band t-shirt, dark makeup, rebellious punk look.
  機能工裝褲 (多袋) | Baggy cargo pants with many pockets, strappy crop top, tactical belt, chains hanging from belt loops, industrial street style.
  別針裝飾外套 | Denim jacket covered in safety pins and patches, customized DIY look, chaotic punk texture, rough edges.
  哥德蕾絲馬甲 | Black lace corset over white shirt, gothic aesthetic, cross necklace, dark red lipstick, vampire vibe.
  透明機能外套 | Transparent PVC rain jacket, worn over neon sports bra, waterproof texture, futuristic streetwear, visible layers.
  反光條連身衣 | Bodysuit with 3M reflective strips, glowing white under flash, sleek tech material, sporty futuristic look.
  戰術胸掛 (Chest Rig) | Tactical chest rig harness, black nylon straps, buckles, worn over a tight bodysuit, techwear fashion.
  袖套/波利羅 (Bolero) | Super cropped shrug (bolero), covering only arms and shoulders, chest completely exposed, futuristic silhouette.
  賽博面罩穿搭 | Wearing a high-tech half-face mask, neon glowing accents, hoodie up, mysterious cyberpunk hacker look.
  數位印花緊身衣 | Tight top with glitch art/matrix print, vivid digital colors, synthetic fabric, cyber-aesthetic.
  挖空連體衣 (Cut-out) | Asymmetric cut-out bodysuit, complex holes revealing cleavage and waist, sleek modern design, solid color (black/nude).
  芭蕾核 (Balletcore) | Satin wrap top and tulle skirt, pink ribbons tied around legs, ballet flats, soft feminine aesthetic, delicate coquette vibe.
  乳膠馬甲外穿 | Shiny latex waist cincher worn over a dress, contrast texture, hourglass figure emphasizing, fetish fashion trend.
  超短開襟衫 (前扣) | Micro cardigan held by one pin, extreme cleavage, barely covering bust, ribbed knit texture, Jacquemus style.
  緞面吊帶裙 (Slip) | Satin slip dress, cowl neckline, thin spaghetti straps, silky texture clinging to body, minimalist 90s chic.
  透視薄紗上衣 | Sheer mesh top with embroidered flowers, visible bra underneath, romantic and edgy mix, tattoo-like effect.
  連體開檔褲 (Chaps) | Denim chaps over bikini bottom, legs covered but crotch and butt exposed/layered, festival fashion, daring cowboy look.
  皮草大衣 + 內衣 | Oversized faux fur coat over lingerie, luxury mob wife aesthetic, bulky coat contrasting naked body, glamorous.
  頸帶連身裙 (Choker Dress) | Dress with attached choker neck, V-neckline connecting to collar, cutout chest, sleek evening wear.

頭配件
  舌環 (伸舌可見) | Silver barbell tongue piercing, visible on extended tongue, shiny metal.
  金屬牙套 (矯正器) | Silver dental braces on teeth, visible when smiling,mouth open, metallic glint.
  鼻中隔環 (公牛環) | Silver septum ring piercing, metallic reflection under nose.
  各種耳骨環 (滿耳) | Multiple silver studs and hoops along the outer ear cartilage, punk aesthetic.
  單眼眼罩 (醫療風) | White medical eye patch covering one eye, straps visible against hair.
  蕾絲蒙眼帶 (半透) | Black lace blindfold covering eyes, skin visible through mesh pattern.
  口球 (環形,開口) | Ring gag (open mouth gag), teeth and tongue visible through the center hole, leather straps.
  口塞 (實心球) | Red silicone ball gag, mouth stretched open wide, cheeks puffed slightly.
  金屬項圈 (帶鎖) | Heavy steel collar with a padlock throat, locked tight against skin.
  皮革項圈 (帶牽繩) | Black leather collar with a metal D-ring, silver chain leash hanging down from it.
  寵物名牌 (掛牌) | Small metal dog tag hanging from choker, resting in the hollow of collarbone.
  頸後條碼紋身 | Black barcode tattoo stamped on the back of the neck, visible under updo hair.
  聽診器 (掛頸) | Medical stethoscope hanging loosely around neck, metal chest piece resting on skin.
上半配件
  乳環 (透視可見) | Silver barbell nipple piercings, visible through sheer,wet fabric, distinct metallic glint.
  乳環 (裸露) | Silver ring nipple piercings, piercing directly through the skin, exposed.
  乳頭夾 (帶鍊) | Metal nipple clamps with rubber tips, connected by a decorative silver chain.
  乳貼 (X型) | Black electrical tape forming an X shape over nipples, skin indented slightly.
  胸下刺青 (曼陀羅) | Intricate mandala tattoo inked on the ribs directly under the breast curve (underboob).
  胸部抹油 (高光) | Skin on chest coated in oil, glistening highlights, sweat droplets running down cleavage.
  緊身衣勒痕 | Red indentation marks on skin from tight clothing,straps, visible after removal.
  肚臍環 (垂墜) | Navel piercing with dangling crystal, midriff exposed, stomach skin shiny.
  魅魔紋身 (子宮紋) | Pink heart-shaped succubus crest tattoo (womb tattoo) on lower abdomen, waistline pulled down to reveal.
  後背抓痕 | Red scratch marks running down the back skin, fresh injury texture.
  繩縛勒痕 (Shibari) | Red rope marks (indentations) remaining on skin pattern, rope removed.
  身體鍊 (Body Chain) | Gold body chain crossing over torso and waist, digging slightly into soft flesh.
  手銬 (金屬) | Stainless steel handcuffs locking wrists together behind back or in front.
  拇指銬 | Small metal thumb cuffs locking thumbs together, hands rendered useless.
  皮鞭 (手持) | Holding a black braided leather riding crop,whip, leather texture.
  穿孔束腰 (Corset) | Corset piercing (flesh laced), ribbon laced directly through skin rings on back/chest.
下半配件
  陰部穿環 (VCH) | Vertical hood piercing (silver barbell), visible between legs, underwear pulled aside.
  恥骨紋身 (淫紋) | Black tribal tattoo just above the pubic area, visible due to low-rise waistline.
  大腿襪勒肉 | Top of thigh-high stockings digging into thigh fat (skindentaion), absolute territory.
  大腿環 (Garter) | Black leather leg garter belt strapped tight around one thigh.
  吊襪帶夾 | Metal suspender clips attaching stockings to panties, stretching the fabric.
  網襪破洞 | Black fishnet stockings with large rips and tears, skin poking through.
  比基尼曬痕 | Distinct white tan lines on hips and chest, contrasting with tanned skin.
  腿內側刺青 | Cursive script tattoo inked on the sensitive inner thigh skin.
  駱駝趾 (Cameltoe) | Fabric of panties,leggings pulled tight between legs, distinct outline visible.
  半脫內褲 | Panties pulled down to knees,ankles, limiting movement, legs spread.
  跳蛋 (凸起) | Small vibrating egg inside panties, creating a visible bulge in fabric, buzzing.
  跳蛋 (控制器) | Holding a remote control for a vibrator, cord trailing into clothing,skirt.
  按摩棒 (使用中) | Pink magic wand vibrator pressed against crotch area, fabric straining.
  肛塞 (寶石尾) | Metal butt plug with a red jewel base, visible between buttocks,under skirt.
  狐狸尾巴 (插入式) | Large fluffy fox tail plug extending from rear, attached to body.
  腳鐐 (鐵鍊) | Heavy iron shackles around ankles, connected by a short chain, limiting stride.
  貞操帶 (外穿) | Metal chastity belt with lock, worn over skin or underwear.
  
全身配件  
  潤滑液 (流淌) | Clear viscous liquid (lube) running down inner thighs, shiny texture.
  全身濕透 (透視) | Clothing completely soaked with water, clinging to skin, becoming transparent (wet look).
  全身塗油 | Entire body coated in baby oil, extreme glossy highlights, slick skin.
  白色液體 (潑灑) | Splashes of white viscous liquid on face and chest, messy texture.
  大量白濁 (顏射後) | Face and hair covered in sticky white viscous liquid, bukkake aftermath, messy dripping texture.
  全身史萊姆 (粘液) | Body covered in translucent green/pink slime, viscous mucus dripping, trapped in slime.
  全身塗油 (極厚) | Skin coated in thick heavy oil, extreme specular highlights, slippery body, glistening under light.
  失禁 (尿漬) | Dark wet patch on crotch area of panties/clothing, embarrassment, liquid dripping down legs.
  身上寫字 (羞辱) | Black marker writing on skin (stomach/face/thighs), humiliating words/marks, graffiti on body.
  小腹隆起 (異物感) | Distended stomach, visible bulge in lower abdomen shape (womb bulge), skin stretched tight.
特級配件 
  雙頭龍 (雙穴) | Double penetration, using double-ended dildo, filled both holes, stretched fabric/skin.
  巨型道具 (比例失調) | Interaction with oversized toy/object, too large for human body, extreme stretching capability.
  產卵/排卵 (暗示) | Translucent tube inserting eggs, or belly filled with round shapes (oviposition), internal texture visible.
  導尿管/管線 | Transparent medical catheter tubes connected to body, infusion bags, clinical hospital setting.
  灌腸 (連接中) | Enema tube inserted her ass, clear hose filled with liquid, bloated stomach sensation.
  電擊貼片 | Electrodes pads attached to nipples and sensitive areas, wires connecting to a control box.
  觸手纏繞 (凌辱) | Multiple slimy tentacles wrapped tightly around limbs and torso, squeezing flesh, suction cups.
  觸手侵入 (衣下) | Tentacles crawling under clothes/skirt, fabric bulging and moving, sneaky assault.
  昆蟲/蠕蟲 | Cockroaches/worms crawling on skin or inside clothes, filth and disgust texture.
  半獸化 (乳牛/母狗) | Cow print bikini/body paint, attached udders, wearing a bell collar, human cattle roleplay.


基本配件
  黑框眼鏡 (文青) | Thick black-rimmed eyeglasses, rectangular frame, intellectual vibe.
  金絲圓框眼鏡 (復古) | Thin round gold-wire glasses, retro aesthetic, oversized lenses.
  無框眼鏡 (專業) | Rimless eyeglasses, clean and sharp look, light reflecting on lenses.
  墨鏡 (戴著) | Dark sunglasses covering eyes, cool and mysterious look.
  半框眼鏡 (眉框) | Browline glasses (semi-rimless), vintage intellectual style.
  雀斑 (自然妝) | Sun-kissed freckles scattered across nose and cheeks, natural skin texture.
  腮紅 (微醺感) | Rosy red blush on cheeks, soft makeup, slightly shy appearance.
  淚痣 (左眼下) | Small beauty mark (mole) under the left eye.
  美人痣 (嘴角) | Small beauty mark (mole) near the corner of the mouth.
  OK繃 (鼻樑) | Small adhesive bandage (band-aid) stuck across the nose bridge.
  OK繃 (臉頰) | Small adhesive bandage (band-aid) on one cheek.
  異色瞳 | Heterochromia eyes (one blue, one green), captivating gaze.
  臉部亮片 (派對) | Glitter makeup scattered on cheekbones, festival style.
  珍珠耳釘 (氣質) | Small white pearl stud earrings, classic and elegant.
  鑽石耳釘 (閃亮) | Tiny diamond stud earrings, subtle sparkle, luxury vibe.
  金屬小圈耳環 | Small gold hoop earrings, simple and minimalist.
  大圓圈耳環 (歐美) | Large silver hoop earrings, fashion statement.
  黑色耳釘 (個性) | Simple black round stud earrings, unisex style.
  耳骨夾 (無耳洞) | Gold ear cuff wrapping around the ear cartilage (no piercing needed).
  流蘇耳環 | Long dangling tassel earrings, swaying gently.
  十字架耳環 | Silver cross pendant earrings, hanging down.
  藍牙耳機 (單耳) | Wireless earbud in one ear, white plastic, tech accessory.
  細金項鍊 | Delicate thin gold chain necklace, minimalist style.
  銀色鎖骨鍊 | Silver necklace with a small pendant resting on collarbone.
  黑色頸帶 (Choker) | Simple black velvet choker ribbon, tight around neck.
  珍珠項鍊 | String of white pearls around neck, vintage elegance.
  耳罩耳機 (掛頸) | Large wireless headphones resting around the neck.
  識別證掛繩 | Lanyard with an ID card holder hanging around neck.
  多層手環 | Stack of beaded and leather bracelets on wrist, bohemian style.
  紅線手鍊 | Thin red string bracelet tied around wrist (lucky charm).
  簡約戒指 | Simple silver band ring on finger.
  紅寶石戒指 | Large gemstone ring on finger, luxury accessory.
  指甲油 (紅色) | Bright red painted fingernails, glossy finish.
  指甲油 (黑色) | Black nail polish, edgy style.
  指甲油 (裸色,法式) | Nude or French manicure, clean and natural nails.


場景
  外科醫師 , 手術刀 , 手術室 | Surgeon, medical scrubs, surgical mask and cap, holding scalpel, bright hospital operating room.
  牙醫助理 , 診療椅 , 診所 | Dental assistant, clinic uniform, dental chair with tools, bright dental office.
  實驗室技師 , 顯微鏡 , 實驗室 | Lab technician, white lab coat, safety goggles, microscope on desk, chemical laboratory.
  女秘書 , 電腦 , 辦公室 | Secretary, office wear, desk with computer and phone, stack of papers, typical office environment.
  CEO女總裁 , 會議桌 , 董事會 | CEO, business suit, long meeting table, leather executive chair, modern boardroom.
  銀行行員 , 點鈔機 , 櫃檯 | Bank teller, bank uniform vest, behind teller counter, cash counting machine, bank hall.
  法官 , 木槌 , 法庭 | Judge, black robe, wooden gavel and sound block on high bench, courtroom.
  女警探 , 警徽手銬 , 現場 | Detective, jacket over clothes, police badge on belt, handcuffs, crime scene tape in background.
  交警 , 反光背心 , 路口 | Traffic officer, police uniform, high-visibility reflective vest, whistle, busy city intersection.
  保全 , 對講機 , 監控室 | Security guard, security uniform, walkie-talkie, wall of CCTV monitor screens, dark room.
  工地監工 , 安全帽圖紙 , 工地 | Construction supervisor, hard hat, safety vest, holding rolled blueprints, active construction site.
  油漆工 , 油漆桶刷 , 裝修房 | Painter, paint-splattered overalls, holding paint roller, open paint bucket, empty room.
  園丁 , 灑水壺 , 溫室 | Gardener, work apron, gloves, holding watering can, glass greenhouse with plants.
  花店店員 , 花剪 , 花店 | Florist, apron, holding floral shears, arranging bouquet, inside flower shop.
  咖啡師 , 義式機 , 吧台 | Barista, coffee apron, operating espresso machine, coffee cup, cafe counter.
  餐廳服務生 , 托盤 , 餐廳 | Waitress, restaurant uniform, holding serving tray with food, dining area with tables.
  女僕 , 雞毛撢 , 豪宅 | Maid, classic maid uniform, holding feather duster, luxury mansion hallway.
  飯店櫃檯 , 電腦 , 大廳 | Hotel receptionist, hotel uniform blazer, behind reception desk with computer, hotel lobby.
  空軍飛行員 , 頭盔 , 機坪 | Fighter pilot, flight suit, holding flight helmet under arm, tarmac next to jet fighter.
  太空人 , 太空裝 , 艙內 | Astronaut, bulky space suit with helmet, control panels, interior of space station.
  游泳選手 , 泳帽泳鏡 , 泳池 | Swimmer, competitive swimsuit, swim cap and goggles, edge of olympic swimming pool.
  體操選手 , 平衡木 , 體育館 | Gymnast, leotard, standing near balance beam, gymnastics arena with mats.
  健身教練 , 啞鈴 , 健身房 | Fitness trainer, sports outfit, holding dumbbell, gym environment with equipment.
  高爾夫球手 , 球桿 , 球場 | Golfer, polo shirt and visor, holding golf club, green golf course with sand trap.
  滑雪者 , 雪仗雪鏡 , 雪山 | Skier, winter snow gear, ski goggles, holding ski poles, snowy mountain slope.
  花式滑冰 , 冰鞋 , 冰場 | Figure skater, skating dress, wearing ice skates, indoor ice rink surface.
  排球選手 , 排球 , 沙灘 | Volleyball player, beach volleyball gear, holding volleyball, sandy beach with net.
  足球員 , 釘鞋足球 , 草地 | Soccer player, team jersey and shorts, wearing cleats, soccer ball at feet, grass field.
  籃球員 , 籃球 , 球場 | Basketball player, basketball jersey, holding basketball, indoor court with hoop.
  拳擊手 , 拳套 , 擂台 | Boxer, boxing shorts, wearing boxing gloves, inside boxing ring with ropes.
  劍道家 , 竹劍護具 , 道場 | Kendo practitioner, kendo uniform (hakama), holding bamboo sword (shinai), wooden dojo.
  擊劍手 , 西洋劍 , 賽道 | Fencer, white fencing suit and mask, holding fencing sword (foil), fencing strip.
  射箭手 , 弓箭 , 靶場 | Archer, arm guard, holding bow and arrow quiver, outdoor archery range with targets.
  賽馬騎師 , 馬鞭 , 馬廄 | Jockey, colorful racing silks, holding riding crop, wooden horse stable.
  攀岩者 , 安全帶 , 岩壁 | Climber, climbing harness and gear, chalk bag, steep indoor rock climbing wall.
  聖騎士 , 盔甲盾牌 , 神殿 | Paladin, shining plate armor, holding sword and heavy shield, stone temple hall.
  盜賊(刺客) , 匕首 , 暗巷 | Rogue, dark leather armor, hooded cloak, holding daggers, dark cobblestone alley.
  死靈法師 , 法杖骷髏 , 墓穴 | Necromancer, dark robes, holding staff with skull, crypt filled with bones.
  德魯伊 , 樹葉裝 , 森林 | Druid, clothing made of leaves and furs, wooden staff, ancient forest setting.
  戰法師 , 魔法書 , 圖書館 | Battlemage, ornate robes, holding open spellbook with glowing runes, magical library.
  野蠻人 , 皮毛斧頭 , 荒原 | Barbarian, fur and leather gear, holding large battle axe, rocky wasteland.
  弓箭手(精靈) , 長弓箭袋 , 森林 | Elf archer, forest tunic, holding longbow and quiver on back, dense woods.
  獸人女戰士 , 粗鐵甲戰鎚 , 營地 | Orc warrior, crude metal armor, holding war hammer, tribal camp with fire.
  龍族混血 , 鱗片 , 火山 | Dragonborn, visible scales on skin, simple armor, volcanic landscape with lava.
  女神 , 絲綢光環 , 虛空 | Goddess, flowing white silk gown, glowing aura, celestial void with stars.
  惡魔女王 , 角與權杖 , 地獄 | Demon queen, dark armor with horns, holding spiked scepter, obsidian throne room.
  森林妖精 , 翅膀 , 花叢 | Fairy, translucent wings, small size, sitting on giant flower in garden.
  樹精(Dryad) ,樹皮皮膚 , 古樹 | Dryad, skin textured like bark and leaves, merging with large ancient tree trunk.
  鍊金術士 , 護目鏡藥水 , 工坊 | Alchemist, apron and goggles, holding bubbling potion flask, cluttered workshop.
  吟遊詩人 , 魯特琴 , 酒館 | Bard, colorful traveling clothes, playing lute instrument, crowded medieval tavern.
  賞金獵人 , 雷射槍 , 酒吧 | Bounty hunter, weathered armor vest, blaster pistol in holster, sci-fi cantina.
  生化改造人 , 機械臂 , 維修台 | Cyborg, visible mechanical arm prosthesis, wires, high-tech repair bay.
  艦長 , 制服徽章 , 艦橋 | Starship captain, fleet uniform with rank badges, spaceship bridge consoles.
  外星大使 , 禮袍 , 議會 | Alien diplomat, ceremonial robes, unique alien features, futuristic council chamber.
  實驗體 , 繃帶管線 , 培養槽 | Lab subject, minimal bandages, attached tubes and wires, inside glass stasis tank.
  反抗軍 , 戰術背心步槍 , 廢墟 | Rebel soldier, ragged tactical gear, holding sci-fi rifle, destroyed concrete ruins.
  駭客 , 帽T筆電 , 機房 | Netrunner, dark hoodie, typing on laptop with cables connected, dark server room.
  機甲駕駛 , 操作服 , 駕駛艙 | Mech pilot, tight plug-suit, inside robot cockpit with screens and joysticks.
  全息投影AI , 發光數據 , 網路 | AI avatar, glowing digital body, floating data streams, abstract cyberspace.
  火星殖民者 , 工作服 , 溫室 | Colonist, utility jumpsuit, tending to plants, inside Mars biodome habitat.
  原始人 , 獸皮石器 , 洞穴 | Cavewoman, animal pelts, holding stone tool, inside rock cave with fire.
  埃及祭司 , 黃金飾品 , 神廟 | Egyptian priestess, linen dress and gold collar, holding ankh symbol, temple interior.
  羅馬貴婦 , 托加長袍 , 庭院 | Roman noblewoman, elegant toga dress, stone benches, marble villa courtyard.
  維京盾女 , 鎖子甲戰斧 , 長船 | Viking shieldmaiden, chainmail shirt, holding round shield and axe, deck of longboat.
  中世紀農婦 , 粗布裙 , 麥田 | Medieval peasant, simple rough dress, holding basket, large wheat field.
  文藝復興貴族 , 絲絨禮服 , 畫廊 | Renaissance lady, ornate velvet gown, standing in palace gallery with oil paintings.
  海盜船長 , 三角帽彎刀 , 甲板 | Pirate captain, pirate coat and hat, holding cutlass sword, wooden ship deck.
  西部牛仔 , 皮套褲左輪 , 酒館 | Cowgirl, hat and leather chaps, revolver gun in holster, wooden western saloon.
  維多利亞寡婦 , 黑紗喪服 , 墓園 | Victorian woman, full black mourning dress and veil, foggy old cemetery.
  爵士女伶 , 麥克風 , 舞台 | Jazz singer, vintage style dress, standing at old microphone stand, club stage.
  戰地護士 , 復古制服紅十字 , 帳篷 | Nurse, vintage WWII uniform with red cross emblem, medical tent with cots.
  家庭主婦 , 圍裙 , 廚房 | Housewife, retro dress and apron, baking mitts, 1950s style kitchen.
  迪斯可舞者 , 喇叭褲 , 舞廳 | Disco dancer, 70s outfit with bell-bottoms, under disco ball reflecting lights.
  韻律操教練 , 緊身衣 , 教室 | Aerobics instructor, 80s spandex outfit and leg warmers, bright gym studio.
  學生 , 制服書包 , 走廊 | Student, school uniform blazer,skirt, holding school bag, school hallway with lockers.
  蘿莉塔 , 洋裝陽傘 , 茶會 | Lolita girl, frilly dress and bonnet, holding lace parasol, elegant tea room.
  傳統哥德 , 黑絲絨十字架 , 教堂 | Trad Goth, black velvet clothing, silver cross necklace, inside old gothic cathedral.
  情緒搖滾(Emo) , 條紋衫吉他 , 臥室 | Emo girl, striped shirt and skinny jeans, electric guitar in background, bedroom with posters.
  嘻哈女孩 , 帽T滑板 , 塗鴉牆 | Hip-hop girl, oversized hoodie and cap, holding skateboard, brick wall with graffiti.
  滑板少女 , 滑板 , 滑板場 | Skater girl, casual clothes, riding skateboard, concrete skate park.
  波希米亞 , 長裙吉他 , 露營車 | Bohemian girl, flowing maxi dress, acoustic guitar nearby, inside decorated vintage van.
  重機騎士 , 皮衣安全帽 , 公路 | Biker girl, leather jacket, holding motorcycle helmet, next to motorcycle on highway.
  銳舞(Rave) , 螢光棒 , 派對 | Raver, bright neon clothing, holding glow sticks, dark warehouse party with lasers.
  文青 , 眼鏡書本 , 書店 | Hipster girl, cardigan and glasses, holding a book, quiet corner of independent bookstore.
  古著控 , 復古大衣 , 市集 | Vintage lover, retro style coat and bag, browsing outdoor flea market stalls.
  大學生 , 帽T筆電 , 講堂 | College student, casual hoodie and jeans, open laptop on desk, large lecture hall.
  宅女(Gamer) , 耳機手把 , 房間 | Gamer, oversized t-shirt, wearing gaming headset, holding controller, dark room with screens.
  網紅 , 手機環形燈 , 網美牆 | Influencer, trendy clothes, holding smartphone, ring light stand in front of decorative wall.
  背包客 , 大背包地圖 , 旅舍 | Backpacker, casual clothes, large hiking backpack nearby, looking at map in hostel.
  家庭主婦 , 推車 , 超市 | Shopper, casual coat, pushing shopping cart, supermarket aisle.
  遛狗者 , 牽繩 , 公園 | Dog walker, raincoat, holding dog leash attached to dog(s), park path.
  畫家 , 沾滿顏料的圍裙 , 畫室 | Artist, paint-stained clothes and apron, holding palette and brush, art studio with easel.
  瑜珈學員 , 瑜珈服 , 客廳 | Yoga practitioner, yoga leggings and top, on yoga mat, sunny living room.
  海灘遊客 , 比基尼墨鏡 , 躺椅 | Sunbather, bikini and sunglasses, lying on beach lounge chair, sandy beach.
  居家少女 , 睡衣枕頭 , 床上 | Girl, comfortable pajamas, surrounded by pillows, messy bed.
  女殺手 , 黑衣手槍 , 酒店 | Assassin, black tactical gear, holding pistol with silencer, dim hotel room.
  黑幫大姐 , 皮草雪茄 , 倉庫 | Gangster boss, luxurious fur coat, holding cigar, abandoned warehouse setting.
  邪教徒 , 長袍匕首 , 祭壇 | Cultist, dark hooded robe, holding ceremonial dagger, stone cellar with altar.
  小偷 , 緊身衣繩索 , 屋頂 | Thief, black tight-fitting outfit, coiled rope over shoulder, night rooftop.
  囚犯 , 橘色囚服手銬 , 牢房 | Prisoner, standard orange jumpsuit, handcuffs on wrists, inside jail cell with bars.
  大型狗籠內 | Inside a large metal wire crate, padded floor mat, view through cage bars.
  手術台 (無菌冷光) | Cold stainless steel operating table, sterile green drapes, surgical light overhead.
  警察偵訊室 (單面鏡) | Dim interrogation room, metal table, single hanging light bulb, one-way mirror.
  精神病院軟墊房 | White padded room, quilted soft walls, straitjacket straps on floor.
  監獄獨居房 | Concrete jail cell, metal bars, narrow bunk bed, toilet in corner.
  地下室地牢 | Stone dungeon walls, iron shackles on wall, damp floor, wooden torture rack.
  標本展示箱 | Inside a large glass display case, museum setting, spotlight from above.
  屠宰場冷凍庫 | Walk-in freezer, hanging plastic strip curtains, frost on metal shelves.
  銀行金庫 | Inside thick steel vault door, stacks of cash and deposit boxes.
  貨運木箱 | Inside a large wooden shipping crate, packing straw, gaps in wood slats.
  滿員電車 (通勤) | Crowded subway car interior, metal grab handles, sliding glass doors, blurred commuters.
  學校講台 | Wooden lecture podium, microphone, empty classroom with blackboard background.
  辦公桌下 | Underneath a heavy wooden executive desk, view of leather chair legs and cables.
  公廁隔間 | Public restroom stall, graffiti on walls, gap under the door, tiled floor.
  更衣室 (試衣間) | Cramped fitting room, heavy velvet curtain, full-length mirror, hooks on wall.
  圖書館書架走道 | Narrow aisle between tall bookshelves, quiet atmosphere, dusty books.
  電梯內 (鏡面) | Inside modern elevator, mirrored walls, metal handrail, floor indicator buttons.
  透明公車候車亭 | Glass bus stop shelter, rain streaking on glass, wet street at night.
  健身房更衣室 | Locker room, rows of metal lockers, wooden benches, steam in air.
  醫院病床 (隔簾) | Hospital bed with white sheets, privacy curtain drawn around, IV drip stand.
  廉價旅館 (霓虹) | Cheap motel room, patterned carpet, flickering neon sign outside window.
  愛情旅館 (圓床) | Love hotel suite, red circular bed, mirrored ceiling, mood lighting.
  水床 | Vinyl waterbed, rippled surface, vintage bedroom decor.
  透明浴室 | Glass-walled bathroom, visible shower head, condensation on glass.
  日式榻榻米房 | Traditional tatami room, sliding paper doors (shoji), futon mattress on floor.
  按摩台 | Massage table with face hole, stack of white towels, dim spa room.
  桑拿房 | Wooden sauna room, hot stones, steam filling the air.
  豪宅落地窗前 | In front of floor-to-ceiling glass window, night city skyline view, sheer curtains.
  露營帳篷內 | Inside small camping tent, sleeping bag, flashlight glow, fabric texture.
  車後座 (皮椅) | Backseat of luxury car, black leather upholstery, seatbelt buckles.
  鏡子屋 (無限反射) | Room lined entirely with mirrors, infinite reflections, bright lights.
  紅色絲絨房 | Room with red velvet walls, heavy red curtains, plush red carpet.
  塑膠布殺手房 | Room covered in clear plastic sheets, taped to walls and floor, clinical lighting.
  廢棄教室 | Abandoned classroom, broken desks piled up, peeling paint, dusty sunlight.
  充滿氣球的房間 | Room filled with hundreds of colorful balloons on floor, party atmosphere.
  牛奶浴缸 | Bathtub filled with opaque white milk, rose petals floating, bathroom tiles.
  充滿油的地面 | Dark room, floor covered in black oil,slime, reflective and slick surface.
  繩縛架 (Shibari) | Wooden suspension frame, ropes hanging down, tatami mats.
  鋼管舞舞台 | Stage with chrome pole, hazy atmosphere, purple spotlights, messy dollar bills.
  攝影棚 (無縫牆) | Professional photo studio, white seamless background (cyclorama), softboxes.
  後巷垃圾箱旁 | Dark alleyway, brick walls, metal dumpster, puddles on ground.
  網咖包廂 | Tiny internet cafe cubicle, leather reclining seat, glowing monitor screen.
  施工鷹架 | Construction scaffolding platform, safety mesh, city lights below.
  投幣式洗衣店 | Laundromat at night, rows of spinning washing machines, fluorescent lights.
  KTV包廂 | Karaoke room, leather sofa, low table with alcohol bottles, disco ball.
  停車場樓梯間 | Concrete stairwell in parking garage, industrial railing, harsh shadows.
  撞球檯上 | On top of pool table, green felt surface, billiard balls scattered.
  賭場牌桌 | Green poker table, stacks of chips, playing cards spread out.
  告解室 | Wooden confessional booth, lattice screen divider, dark interior.
  屋頂水塔旁 | Rooftop access, metal water tank, rusty ladder, open sky.
  體育倉庫 (跳箱,軟墊) | Inside gym storage room, stacks of gym mats, wooden vaulting box, balls in baskets, dusty air.
  保健室 (隔簾) | School infirmary, white privacy curtain drawn around, narrow bed, medicine cabinet.
  頂樓 (鐵網) | School rooftop, chain-link fence, concrete floor, open sky,sunset background.
  放學後的空教室 (夕陽) | Empty classroom at sunset, rows of wooden desks and chairs, blackboard with chalk writing, orange light.
  樓梯轉角 (平台) | Stairwell landing, concrete steps, metal railing, fire extinguisher, echoey atmosphere.
  視聽教室 (暗房) | Audio-visual room, heavy blackout curtains, projector screen, tiered seating, dark ambiance.
  圖書館角落 (死角) | Quiet library corner, tall wooden bookshelves blocking view, rolling ladder, dust motes.
  校舍後方 (焚化爐,樹蔭) | Behind the school building, secluded spot near trees, old incinerator, shadows.
  理科實驗室 (人體模型) | Science laboratory, black stone counters, anatomical skeleton model, glass beakers and test tubes.
  音樂教室 (鋼琴) | Music room, grand piano in center, portraits of composers on wall, afternoon light.
  美術教室 (石膏像) | Art room, wooden easels, white plaster busts,statues, paint-stained floor, messy canvas.
  家政教室 (廚房) | Home economics room, stainless steel kitchen counters, sink, cooking utensils.
  廣播室 (麥克風) | Broadcasting room, mixing console, microphone on stand, "On Air" light.
  學生會辦公室 (權力感) | Student council room, long meeting table, fancy high-back chair, tea set.
  女子更衣室 (置物櫃) | Girls' locker room, rows of metal lockers, wooden bench in center, tiled floor.
  室內泳池 (夜晚水光) | Indoor swimming pool at night, water reflecting moonlight, lane markers, tiled deck.
  網球場邊 (鐵絲網) | Tennis court sideline, green court surface, chain-link fence, umpire chair.
  弓道場 (木地板) | Traditional archery dojo (Kyudo), polished wooden floor, sliding doors, open to garden.
  鞋櫃區 (玄關) | School entrance, rows of shoe lockers (getabako), sunlight form entrance doors.
  教職員辦公室 (老師桌) | Faculty office, cluttered teacher's desk, stacks of homework papers, coffee mug.

胸部姿勢
  前傾，胸部重力垂墜。 | Low angle shot, her heavy breasts hang down naturally in a teardrop shape as she leans her upper body deep toward the camera.
  仰躺，胸部向兩側攤開。 | Top-down view, her bust spreads sideways across her chest due to gravity as she lies flat on her back.
  側向，胸部從腋下溢出。 | Side profile, her full breasts overflow from the sides under her armpits, showing a massive rounded curve.
  俯臥，胸部受體重壓扁。 | Surface level shot, her breasts are flattened against the floor by her body weight, spreading outward in a soft, squashed shape.
  四足，胸部隨動作晃盪。 | Worm's eye view, her heavy bust sways side to side between her arms with a sense of weight as she moves on all fours.
  深吸氣，胸口極度隆起。 | Close-up, her chest rises high with a deep breath, creating strong tension between her collarbones and full bust.
  特寫，下胸重力褶痕。 | Underboob macro, the heavy weight of her breasts creates a deep, curved skin fold at the base where they meet her torso.
  抓握，指尖深陷變形。 | Macro shot, her fingers sink deep into the soft flesh of her breasts, creating realistic indentations from the pressure.
  托舉，胸部頂端繃圓。 | Close-up, she cups and lifts her breasts from below, forcing the upper curves into a firm, rounded shape.
  內推，胸部強力中夾。 | Chest close-up, she pushes her breasts together firmly, creating an incredibly deep and tight vertical cleavage.
  抓揉，肉量指縫溢出。 | Tight close-up, her hand tries to grab her breast, but the soft flesh overflows through her fingers.
  頂部按壓，肉量外擴。 | High angle, she presses down on her breasts with her palms, causing the flesh to spread sideways in a soft deformation.
  交叉勒胸，側乳內推。 | Frontal view, she crosses her arms tightly over her chest, her palms pressing into her side-boobs to maximize the lift.
  手蓋胸，指縫間窺視。 | Intimate close-up, she covers her breasts with her hands, but her spread fingers reveal glimpses of the skin beneath.
  交替揉弄，組織位移。 | Action shot, her hands knead her soft breasts, showing realistic shifting and rippling of the flesh.
  指尖拉扯，皮膚張力。 | Macro, her fingertips pinch and pull the skin of her breast, showing high elastic tension.
  極低角，胸部壟斷畫面。 | Extreme low angle, her massive bust looms over the lens, dominating the frame and obscuring her chin.
  俯拍，胸部推至鎖骨。 | High angle POV, her breasts are pushed all the way up to her collarbones, filling the entire neckline area.
  胸溝夾物，厚度壓迫。 | Close-up, she holds a bottle tightly within her deep cleavage, emphasizing the crushing thickness of her bust.
  肘夾，胸部極前挺。 | Frontal view, her elbows squeeze inward against her sides, thrusting her chest forward into an extreme projection.
  微距，胸肉碰觸交點。 | Extreme macro, focusing on the exact point where her breasts meet, showing the intense pressure and fleshiness.
  跪姿，雙手托胸前呈。 | Point of view (POV), she kneels and offers her breasts to the camera, pushing them forward with both hands.
  背後環抱，勒胸隆起。 | Rear 3/4 view, an arm wraps around her chest from behind, squeezing her bust upward and forward.
  拱背仰躺，胸部頂點。 | Low side angle, she arches her spine while lying back, thrusting her chest high as gravity pulls at the sides.
  雙臂夾擊，一字深溝。 | Close-up, she uses only her upper arms to squeeze her breasts together, creating a tight vertical cleavage line.
  蹲姿，胸部壓在膝蓋。 | Low angle, she crouches so her breasts rest on her knees, the soft flesh being pushed up by her legs.
  扭轉，側乳受臂擠壓。 | Profile view, her torso is twisted, causing one breast to be squashed against her arm in an exaggerated curve.
  交叉擋點，強調側乳。 | Side angle, she crosses her arms to cover herself, focusing the viewer's eye on the side-boob curves.
  咬衣上提，下胸露弧。 | Low angle close-up, she bites her shirt hem and pulls up, revealing the heavy curves of her underboob.
  外撥領口，胸中起伏。 | Wide frontal view, she pulls her collar outwards with both hands, stretching the chest area to reveal inner curves.
  撩衣瞬間，肉量彈出。 | Action freeze, her breasts are captured mid-action as they pop out from her top, showing full rounded volume.
  側臥，手臂托胸位移。 | Side view, she lies on her side with her arm tucked under her bust, shifting the weight upward in a soft curve.
  臂後展，胸肌緊繃高聳。 | Frontal 3/4 view, she stretches her arms far back, pulling the skin taut and making her chest stand out firmly.
  指尖滑過，表面波動。 | Soft focus close-up, her fingertips graze the edge of her breast, showing the sensitive reaction of the soft tissue.
  動態跳動，組織震顫。 | Motion blur, showing the dynamic jiggle and ripple of her heavy breasts as she moves quickly.

臀部姿勢
  俯臥下腰，臀部高聳。 | High angle from behind, she lies prone and arches her lower back deeply, lifting her rounded hips high.
  仰躺，臀部受壓攤平。 | Top-down view, her buttocks spread sideways against the floor as she lies on her back, showing soft expansion.
  臀腿交界，深溝特寫。 | Macro, focusing on the gluteal fold, showing the deep crease where her heavy buttock meets her thigh.
  坐姿，臀部受重攤開。 | Floor level shot, she sits on the floor, her buttocks flattened and spreading outward under her weight.
  四足跪姿，臀部搖晃。 | Rear low angle, on all fours, her hips sway rhythmically with a sense of weight and motion.
  抓臀，指尖深陷肉中。 | Macro, her hands grip her buttocks firmly, her fingers sinking deep into the soft, fleshy tissue.
  內推，臀溝深線特寫。 | Rear view close-up, she pushes her buttocks together with both hands, creating an extreme vertical line.
  併腿，臀內組織擠壓。 | Rear view, she presses her thighs together tightly, causing her inner buttocks to squeeze against each other.
  掌壓，臀肉外溢變形。 | Close-up, her palm presses hard against her glute, showing the soft flesh spreading under the pressure.
  手撥臀部，中心張力。 | Rear POV, she pulls her buttocks apart with her hands, stretching the skin to reveal the inner curves.
  跪坐，臀部壓在小腿。 | Side low angle, she kneels so her buttocks rest on her calves, the flesh being pushed up and flattened.
  扭腰，單側臀部弧線。 | Profile view, her torso is twisted, creating high tension in one hip while the other remains soft.
  抬腿，臀部肌肉收縮。 | Rear 3/4 view, she raises one leg high, the gluteal muscle stretching and tightening under the skin.
  塌腰，臀部後向突出。 | Side profile, kneeling with an arched back, her buttocks thrusting upward and back in a bold curve.
  前傾，臀部緊繃圓弧。 | Rear view, she leans far forward, pulling her buttocks taut into a large, rounded and firm silhouette.
  地平視角，臀部壟斷。 | Extreme low angle, her hips loom over the camera lens, creating an immense sense of volume.
  側躺，下臀受壓溢出。 | Floor level side view, lying on her side, the bottom hip is squashed while the top hip forms a perfect S-curve.
  行走，臀部波動位移。 | Action shot, showing the dynamic swaying and rippling of her hips as she shifts her weight.
  指劃臀部，微小形變。 | Macro, her fingertip grazes the edge of her buttock, causing a subtle indentation on the soft skin.
  臀部交匯，深邃陰影。 | Tight close-up, the meeting point of her buttocks, showing extreme fleshiness and deep shadows.

腿部姿勢
  併腿坐，大腿肉擠壓。 | Seated front view, her legs are crossed tightly, the soft flesh of her thighs pressing together hard.
  坐姿，大腿重力攤平。 | High angle, her thighs spread outward as she sits, showing the soft expansion of the leg muscles.
  M型腿，內側拉扯張力。 | Frontal low angle, M-shaped leg pose, showing the intense stretch and tension in her inner thighs.
  側躺併腿，下腿受壓。 | Side view, her legs are stacked as she lies on her side, the bottom thigh flattened by the top leg.
  夾膝，大腿外側隆起。 | Frontal view, she squeezes her knees together, causing the flesh of her upper thighs to bulge outward.
  捲趾，足弓緊繃弧度。 | Foot macro, her toes are curled tightly inward, showing a high foot arch and intense muscle tension.
  張趾，腳趾間隙特寫。 | Foot close-up, her toes are spread wide apart, revealing the space between them and the foot structure.
  墊腳，小腿肌群收縮。 | Side low angle, she stands on her tiptoes, her calf muscles tensed and defined in an elegant line.
  足底相對，足弓對稱。 | Bottom view, the soles of her feet are pressed together, toes slightly curled in a symmetrical pose.
  繃足，腿部直線延伸。 | Long shot, her leg is fully extended with pointed toes, creating a long, continuous line of tension.
  足蹭腿，觸碰連動。 | Leg close-up, one foot rubs against the other calf, showing the interaction between the toes and skin.
  抓腿，指尖深陷大腿。 | Macro, her hand grips her thigh, fingers sinking deep into the soft muscle and skin.
  W型坐，大腿內側放鬆。 | High angle, W-shape seating, knees together and feet spread, showing relaxed inner thigh lines.
  指滑腿側，內側延伸。 | Low angle tracking, her fingertips graze her inner leg from the ankle up to the thigh.
  趾勾拉扯，足部張力。 | Foot close-up, her toes hook onto a hem and pull, showing the functional tension in her foot.

體位姿勢
  肘撐貓式，臀部朝天。 | Rear 3/4 low angle, kneeling on her elbows with a deep spinal arch, her hips pointed straight up.
  犁式，腿過頭臀部對向。 | Frontal low angle, her legs are folded over her head, toes touching the floor, hips elevated.
  手勾膝，開腿完全開放。 | Frontal POV, lying on her back, she hooks her hands behind her knees and pulls them wide.
  跪姿反折，胸腹前挺。 | Side profile, kneeling and leaning far back, hands on her ankles, her chest and stomach thrusting out.
  跨坐，重心壓迫跨部。 | High angle POV, she straddles and presses her full weight down, her hips spreading wide and soft.
  站立前摺，臀部後突。 | Side view, standing with torso folded down against her legs, her hips protruding back sharply.
  側臥抬腿，極大夾角。 | Frontal view, lying on her side, she pulls her top leg high toward her shoulder in an extreme split.
  臀橋，腰臀高挺。 | Low side angle, a glute bridge pose, her hips pushed high, stomach pulled taut and flat.
  W跪坐，重心沉底。 | High angle, W-shape kneeling with hips touching the floor, inner thighs relaxed and soft.
  四足扭轉，乳臀連動。 | Dynamic angle, on all fours with a twisted torso, her chest and hip curves reacting to the torque.
  仰臥蝴蝶式，跨部開放。 | Top-down view, lying on her back, soles together, knees dropped wide to the sides, hips open.
  球形抱膝，背臀繃圓。 | Side view, knees tightly pressed to her chest, arms wrapping her legs into a compact, rounded ball.
  趴臥後折，脊椎反屈。 | Side low angle, prone with legs bent back, feet near her hips, showing a deep back bend.
  深蹲開腿，重心降底。 | Frontal low angle, deep squat on tiptoes, elbows pushing her knees wide apart, hips sinking low.
  跪步扭腰，跨部拉伸。 | Low angle, lunge position with a twisted torso, stretching the groin and pelvic area.
  後撐式，胸腹向上挺。 | Side 3/4 view, seated with hands bracing behind, torso arched upward, high chest projection.
  腿勾頸後，肢體位移。 | Frontal view, lying on her back, one leg hooked behind her neck, showing extreme body flexibility.
  側跪伸腿，圓臀偏移。 | Frontal 3/4 view, kneeling on one knee with the other leg extended far out, weight on one hip.
  獵食者，腰凹臀高。 | Low angle front, crawling pose, knees off the ground, deep spinal arch, hips held high.
  仰臥拉腿，V型開展。 | Top-down view, lying on her back, hands grabbing her toes, legs pulled wide into a V-shape.
  側撐扭腰，臀正對鏡頭。 | Rear 3/4 view, side plank with a twisted waist, her back and buttocks facing the lens.
  眼鏡蛇式，腹部拉長。 | Side profile, cobra pose, upper body pushed up, stomach stretched, lower back arched deeply.
  反抓踝，身體成弓。 | Side view, bow pose, hands pulling ankles upward from behind, body arched in a tight curve.
  邊緣後垂，胸部重力。 | Side low angle, leaning back off a surface, torso hanging upside down, gravity pulling at the bust.
  X型大字，全身張力。 | Top-down view, sprawled in an X-shape, limbs extended to the limit, torso long and stretched.
  腿折肩上，大腿壓胸。 | Low angle POV, her hips are lifted high while her legs are folded back over her shoulders, her thighs pressing against her chest.
  下腰跪姿，胸腹凸起。 | Side profile, she arches back while kneeling into a deep curve, pushing her chest and stomach forward.
  全拱橋，腹平胸高。 | Low angle, a full bridge pose, her stomach is pulled flat and her chest is thrusting upward.
  坐姿勾腿，胯部拉扯。 | Frontal view, sitting with legs spread wide, she pulls one leg up and hooks her knee over her shoulder.
  後折伏地，正面全張。 | Side view, sitting on her heels, she bends her torso all the way back until her head touches the floor, stretching her front.


特別場景
  聖安德烈十字架 (X架) | Wooden St. Andrew's cross frame, full body strapped spread-eagle to the X-shape, limbs pulled wide, fully exposed frontal view.
  垂直立式綁架 (一字) | Vertical suspension frame (single post), arms tied straight up above head, body stretched long against the post, standing on tiptoes.
  木製枷鎖 (頭手被困) | Head and wrists locked into a wooden pillory frame (stocks), body bent over horizontally, vulnerable rear view.
  金屬展示架 (圓形/輪盤) | Large metal bondage wheel/ring, body spread out and tied to the circumference, rotating frame, symmetrical shot.
  八爪椅 (強制開腿) | Specialized sex chair with mechanical stirrups, legs forced extremely wide apart by metal arms, pelvis thrust upwards, low angle looking between legs.
  醫療束縛床 (冷金屬) | Stainless steel medical table, body strapped flat tightly with heavy leather belts across torso and limbs, sterile top-down view.
  倒吊床 (完全顛倒) | Strapped to an inversion table tilted completely upside down, ankles secured, hair hanging loose towards floor, disorienting angle.
  婦科檢查台 (架高) | Vintage gynecological exam table, legs raised high in metal stirrups, exposed crotch area, clinical lighting.
  天花板單點懸吊 | Single heavy-duty rope hanging from ceiling hook, body suspended mid-air by wrists (strappado), feet dangling off floor.
  複雜龜甲縛懸吊 | Full body intricate Shibari rope bondage, suspended horizontally from multiple ceiling points, floating mid-air, aesthetic rope patterns.
  橫樑/門框懸掛 | Ropes thrown over an exposed wooden beam/door frame, arms tied up, body hanging slightly, straining against gravity.
  繩梯/吊環 | Hanging from rope ladder or gymnastic rings by hands or ankles, body twisted in suspension.
  金屬擴腿桿 (腳踝) | Metal spreader bar locked between ankles, forcing legs wider than natural, lying on back, camera focused on strained inner thighs.
  手腕擴張桿 (背後) | Rigid bar locked between wrists behind the back, elbows forced outward, chest thrust forward, restricted arm movement.
  大型人體籠 (狗籠) | Inside a cramped metal cage large enough for a human, curled up naked on metal floor, looking out through bars.
  拘束衣 (軟墊房) | Wearing a canvas straitjacket with extra long sleeves tied behind back, sitting against padded wall, restricted upper body.
  木馬 (三角架) | Straddling a sharp wooden horse (triangle torture beam), weights hanging from ankles pulling down, painful expression.
  小馬具全套 (口銜) | Wearing full leather pony harness and a metal bit gag with reins attached, crawling on hands and knees, side profile.
  電動木馬/鞍具 | Sitting on a mechanical saddle machine (Sybian), vibrating motion blurred, gripping handles tightly, ecstatic expression.
  真空床/乳膠袋 | Sealed inside a full-body latex vacuum bed/bag, material sucked tight against skin outlining body shape, breathing tube visible.
  機械炮機 (Thrusting Machine) | Large mechanical thrusting machine with dildo attached, piston moving rapidly, body positioned to receive, blurred motion of machine.
  榨乳機 (連接管線) | Connected to an electric milking machine, suction cups attached to nipples/genitals, clear tubes filling with fluid, medical setting.
  刑罰長凳 (Spanking Bench) | Bent over a padded spanking bench, hips raised high, heavy straps securing torso and ankles, vulnerable rear view target.
  籠椅 (椅下囚禁) | Sitting inside a small cage built into the base of a throne chair, looking up through bars at the seat above, cramped posture.
  保鮮膜木乃伊 (Mummification) | Entire body tightly wrapped in layers of clear plastic wrap (cling film), shiny texture, immobile, leaning against wall.
  肉鉤懸吊 (皮帶模擬) | Suspended from heavy industrial hooks attached to a leather back harness, feet off ground, heavy chains, industrial warehouse.
  狗屋 (戶外/室內) | Crawling halfway out of a wooden dog house, wearing a collar and leash, on hands and knees, pet play scenario.
  審問電椅 (仿刑具) | Strapped into a vintage electric chair, heavy leather straps on wrists and ankles, metal headpiece, dramatic lighting.
  玻璃展示櫃 (狹窄) | Contorted inside a small transparent glass cube, body pressed against glass walls, condensation, museum display feel.
  鐵處女 (內部視角) | Standing inside an open Iron Maiden torture device, spikes visible on the interior door, gothic metalwork, claustrophobic.

表情
  無表情/冷漠 | Expressionless, cold stare, poker face, flat gaze.
  溫柔微笑 | Gentle smile, soft expression, friendly vibe.
  害羞臉紅 | Embarrassed, blushing heavily, cheeks flushed red, avoiding eye contact.
  驚訝/張嘴 | Surprised, mouth slightly open (O-shaped), eyebrows raised.
  憤怒/瞪視 | Angry, furrowed brows, intense glare, clenching teeth.
  嫌棄/鄙視 | Disgusted look, looking down on viewer, sneering.
  悲傷/含淚 | Tearing up, eyes welling with tears, sad eyebrows, quivering lips.
  哭泣/流淚 | Crying, tears streaming down face, red puffy eyes, emotional.
  閉眼/享受 | Eyes closed, peaceful expression, enjoying the sensation.
  單眼眨眼 | Winking, one eye closed, playful smile.
  誘惑微笑 | Seductive smile, smirk, corner of mouth raised, confident look.
  吐舌 (俏皮) | Tongue out, sticking tongue out slightly, playful/teasing expression.
  咬嘴唇 | Biting lip, teeth biting lower lip, suppressed desire/nervousness.
  舔嘴唇 | Licking lips, tongue sweeping across upper lip, hungry expression.
  張嘴/邀請 | Parted lips, mouth slightly open, heavy breathing, waiting for something.
  眼神上雷 (翻白眼) | Rolling eyes, looking up intensely, showing whites of eyes.
  眼神迷離 | Hazy eyes, half-open eyelids, unfocused gaze, drunk on atmosphere.
  索吻 | Puckered lips, lips pushed forward ready for a kiss, eyes closed.
  口水拉絲 | Saliva bridge, connecting upper and lower lips, mouth open, glossy liquid texture.
  口紅暈開 | Smeared lipstick, red makeup messy across cheek/chin, aftermath of kissing.
  嘴角流口水 | Drooling, saliva dripping from corner of mouth, loss of control.
  滿臉通紅 (喘息) | Heavy breathing, sweaty face, flushed skin, mouth open panting.
  睫毛膏暈開 (哭後) | Runny mascara, black tear streaks down cheeks, messy makeup.
  臉部沾精 (潑灑) | Cum on face, white viscous liquid splashes on skin/lips, messy.
  含著液體 | Mouth full, cheeks slightly puffed, holding liquid inside mouth.
  吞嚥動作 | Swallowing, throat straining, head tilted back slightly.
  凌亂髮絲遮臉 | Hair sticking to face, sweaty strands covering eyes/mouth, messy look.
  阿黑顏 (經典) | Ahegao, eyes rolled back, tongue sticking out far, double V-sign hands.
  愛心眼 (痴迷) | Heart-shaped pupils, pink glowing hearts in eyes, obsessed expression.
  壞掉的眼神 (無光) | Dead eyes, empty pupils with no highlights, mind break, mouth hanging open.
  催眠眼 (螺旋) | Hypnotized, spiral pattern in eyes, blank smiling face.
  高潮臉 (極樂) | Orgasmic face, mouth wide open in silent scream, eyes rolled back, flushed heavy blush.
  眼皮撐開 (強制) | Eyelids forced open, unable to blink, wide staring eyes, tearing up.
  失神 (張嘴) | Jaw dropped, slack jaw, completely spaced out, unhinged.
  痛苦/忍耐 | Painful expression, frowning heavily, tears in eyes, sweating.
  瘋狂大笑 | Crazy laugh, mouth wide open, manic expression, yandere vibe.
  口交動作 (吸吮) | Sucking shape, cheeks hollowed, mouth wrapped around invisible object.
  像是在吹氣 | Blowing, lips pursed to blow air.
  牙齒打顫 | Chattering teeth, mouth slightly open, shivering expression.
  露出獠牙 (魅魔) | Showing fangs, mischievous grin, sharp teeth visible.
  金屬牙套笑 | Braces, smiling widely to show silver dental braces.
  舌環展示 | Tongue extended showing piercing, silver barbell on tongue.
  咬手指 | Biting own finger/thumb, seductive or nervous.
  空氣吻 | Blowing a kiss, hand near mouth, winking.
  臉頰鼓起 (生氣/吃) | Puffed cheeks, pouting childishly or eating.



髮色
  藍調黑髮 | jet black hair with subtle blue undertones
  午夜光澤烏黑髮 | silky raven black hair with midnight sheen
  暖光栗棕髮 | deep chestnut brown hair with warm highlights
  金點焦糖棕髮 | rich caramel brown hair with golden flecks
  奶油蜜金髮 | honey blonde hair with soft buttery tones
  冰光鉑金髮 | platinum blonde hair with icy shimmer
  玫瑰草莓金髮 | strawberry blonde hair with rosy glow
  銅光紅褐髮 | auburn red hair with copper reflections
  琥珀火銅紅髮 | fiery copper red hair with amber accents
  銀絲灰髮 | soft ash gray hair with silver strands
  星光藍黑髮 | midnight blue-black hair with starry highlights
  珍珠薰衣草調髮 | subtle lavender-tinted hair with pearl sheen
  日吻沙米髮 | natural sandy beige hair with sun-kissed ends
  摩卡深咖啡棕髮 | glossy espresso brown hair with mocha depth
  霧灰銀金髮 | pale silver blonde hair with misty gray undertones
  酒紅桃花心木紅髮 | rich mahogany red hair with burgundy hints
  榛果暖胡桃棕髮 | warm walnut brown hair with hazelnut glow
  蜂蜜金小麥金髮 | golden wheat blonde hair with honeyed waves
  香料肉桂紅髮 | soft cinnamon red hair with spice flecks
  煙灰灰褐髮 | cool taupe gray hair with smoky sheen
  紫靛藍黑髮 | dark indigo-black hair with violet reflections
  花瓣玫瑰調金髮 | light rose-tinted blonde hair with petal shine
  焦糖漩渦太妃棕髮 | earthy toffee brown hair with caramel swirls
  氣泡香檳金髮 | shimmering champagne blonde hair with fizz glow
  橘味薑紅髮 | vibrant ginger red hair with orange zest
  鋼灰石板灰髮 | muted slate gray hair with steel accents
  黑曜石黑髮 | deep obsidian black hair with obsidian polish
  黎明琥珀金髮 | sunlit amber blonde hair with dawn highlights
  秋葉栗紅褐髮 | chestnut auburn hair with autumn leaf tones
  月光錫灰髮 | soft pewter gray hair with moonlight shimmer
  黃昏海軍藍黑髮 | twilight navy-black hair with dusk hints
  杏桃調髮 | delicate peach-tinted hair with apricot blush

髮型
  長直髮軟翻尾 | long straight hair cascading down back with soft flips
  鬆散波浪肩髮 | loose wavy hair falling over shoulders in gentle ripples
  高馬尾緞帶晃 | sleek high ponytail swaying gently with ribbon accents
  亂髮束臉絲 | messy bun with loose strands framing face and tendrils
  肩波波頭層尾 | shoulder-length bob with soft curls and layered ends
  長層自然羽尾 | long layered hair with natural volume and feathered tips
  雙辮花織胸 | twin braids dangling down chest with woven flowers
  側劉海不對稱層 | side-swept bangs over one eye with asymmetrical layers
  優雅盤髮珠針絲 | elegant updo with tendrils escaping and pearl pins
  精靈刺劉海紋 | pixie cut with textured spikes and soft fringes
  半紮波浪蝴蝶結 | half-up half-down with soft waves and bow ties
  長馬尾緞帶蓬根 | long ponytail with ribbon tie and voluminous roots
  海灘波浪鹽感 | beachy waves tumbling freely with salty texture
  法式後辮交織 | french braid along back with interlaced strands
  波波鈍尾內捲 | bob with blunt ends and subtle inward curls
  鬆髻頸絲逃 | loose chignon at nape with escaping wisps
  魚尾肩辮鬆拉 | fishtail braid draping over shoulder with loose pulls
  頂結纏夾飾 | top knot with wrapped strands and decorative clips
  長直中分臉層 | long straight with middle part and face-framing layers
  捲肩彈圈側分 | curly lob with bouncy ringlets and side part
  荷式皇冠織飾 | dutch braid crown with woven accents
  低馬尾扭邊蓬 | low ponytail with twisted sides and soft volume
  蓬亂層羽劉海 | shaggy layers with feathered bangs
  太空包閃針鬆捲 | space buns with glittery pins and loose curls
  瀑布辮波浪落 | waterfall braid cascading down with soft waves
  滑低髻中分平邊 | sleek low bun with center part and smooth sides
  長簾劉海淡提 | long with curtain bangs and subtle highlights
  復古勝利捲針 | retro victory rolls with pinned curls
  不對稱波波斜邊 | asymmetrical bob with angled edges
  繩扭馬尾編細 | rope twist ponytail with braided details
  鬆側辮拉絲 | loose side braid with pulled-out strands
  蓬吹大波根抬 | voluminous blowout with big waves and root lift

瞳色
  深黑瞳微光澤 | deep black pupils with subtle reflective sheen 
  暖巧克力棕瞳金點 | warm chocolate brown pupils with soft golden flecks
  榛棕瞳土調 | hazel brown pupils with natural earthy tones
  琥珀瞳蜂蜜暖光 | soft amber pupils with honeyed warm glow
  海藍瞳藍寶深 | ocean blue pupils with sapphire depth
  冰藍瞳銀底 | icy blue pupils with silver undertones
  翡翠綠瞳葉光 | emerald green pupils with leafy shine
  森林綠瞳苔點 | forest green pupils with mossy hints
  風暴灰瞳雲漩 | stormy gray pupils with cloudy swirls
  石板灰瞳鋼點 | slate gray pupils with steel accents
  紫瞳紫水晶光 | violet purple pupils with amethyst glow
  薰衣草紫瞳花瓣澤 | lavender purple pupils with petal sheen
虹膜  
  虹膜放射纖維 | iris with subtle radial fibers radiating outward
  虹膜細斑點 | iris with fine speckled patterns and soft spots
  虹膜層環紋 | iris with layered ring textures and gentle depth
  虹膜波蕾絲邊 | iris with wavy lace-like edges and natural veining
  虹膜星爆紋 | iris with starburst patterns and faint bursts
  虹膜金斑點 | iris with flecked gold specks and subtle mottling
  虹膜漸層環 | iris with smooth gradient rings and blended hues
  虹膜隱紋摺 | iris with cryptic crypt patterns and hidden folds
眼睫毛  
  長自然捲睫 | long naturally curved lashes with soft upward curl
  中長體積扇睫 | medium length lashes with subtle volume and fan spread
  輕羽毛空氣睫 | wispy lashes with light feathering and airy tips
  厚簇群睫 | thick clustered lashes with natural grouping
  直睫長變 | straight lashes with gentle length variation
  捲彎均距睫 | curled lashes with soft bend and even spacing
眼妝  
  輕煙燻柔融 | subtle smoky eye with soft blending and natural shadow
  水光眼線薄澤 | dewy eyeliner with glossy finish and thin line
  自然睫膏長捲 | natural mascara with light lengthening and curl
  柔閃影微金光 | soft shimmer shadow with faint metallic glow
  翼線潔邊 | winged liner with subtle wing and clean edge
  裸影霧融暖底 | nude eyeshadow with matte blend and warm undertone
唇狀  
  心形唇柔曲 | soft heart-shaped lips with gentle curve
  豐滿唇自然噘 | full plump lips with natural pout
  細優雅唇微弓 | thin elegant lips with subtle bow
  弓形唇峰明 | bow-shaped lips with defined peaks
  圓柔唇均豐 | rounded soft lips with even fullness
  微下唇自然表 | slightly downturned lips with natural expression
唇色  
  自然粉唇微染 | natural pink lips with subtle tint
  玫瑰紅唇柔暈 | rosy red lips with soft blush
  裸米唇霧面 | nude beige lips with matte finish
  漿果紫唇微澤 | berry purple lips with faint sheen
  珊瑚橘唇暖光 | coral orange lips with warm glow
  透明亮唇光澤 | clear glossy lips with transparent shine
唇感  
  水潤唇輕濕 | moist dewy lips with light hydration
  霧面絨唇 | matte soft lips with velvety texture
  光澤亮唇反射 | glossy shiny lips with reflective surface
  豐潤唇微體 | plump hydrated lips with subtle volume
  乾自然唇細線 | dry natural lips with faint lines
  染亮唇柔澤 | tinted glossy lips with soft sheen
"""

In [10]:
# ==========================================
# 2. 轉換邏輯
# ==========================================

def parse_and_convert():
    categories = []
    # 建立對照表：透過 Label 找到 Slug (因為 Content 區塊通常使用 Label 做標題)
    label_to_slug = {} 
    
    # --- 1. 解析 Config ---
    print("正在解析設定...")
    lines = RAW_CONFIG.strip().split('\n')
    for idx, line in enumerate(lines):
        if not line.strip(): continue
        parts = line.split('|')
        if len(parts) < 2: continue
        
        slug = parts[0].strip()
        label = parts[1].strip()
        tags = parts[2].strip() if len(parts) > 2 else ""
        
        # 預設值
        cat_type = 'required' 
        min_count = 1
        max_count = 1
        
        # 邏輯判斷
        is_opt = 'opt' in tags
        mul_match = re.search(r'mul:(\d+)', tags)
        
        if mul_match:
            cat_type = 'group'
            max_count = int(mul_match.group(1))
            min_count = 0 if is_opt else 1
        elif is_opt:
            cat_type = 'optional'
            min_count = 0
            max_count = 1
        
        categories.append({
            'slug': slug,
            'label': label,
            'type': cat_type,
            'min_count': min_count,
            'max_count': max_count,
            'sort_order': idx + 1
        })
        
        label_to_slug[label] = slug
        # 為了保險，也把 Slug 對應到 Slug (防止 Content 標題直接寫 Slug)
        label_to_slug[slug] = slug

    # --- 2. 解析 Content ---
    print("正在解析內容...")
    prompts = []
    content_lines = RAW_CONTENT.strip().split('\n')
    current_cat_slug = None
    
    for line in content_lines:
        line = line.strip()
        if not line: continue
        
        # 判斷是否為標題 (沒有 | 符號，且長度較短，通常是類別名稱)
        if '|' not in line and len(line) < 20:
            if line in label_to_slug:
                current_cat_slug = label_to_slug[line]
            else:
                print(f"警告：找不到對應的類別 '{line}'，跳過此區塊。")
                current_cat_slug = None
            continue
            
        if current_cat_slug and '|' in line:
            parts = line.split('|')
            if len(parts) >= 2:
                p_label = parts[0].strip()
                p_value = parts[1].strip()
                prompts.append({
                    'category_slug': current_cat_slug,
                    'label': p_label,
                    'value': p_value
                })

    # --- 3. 生成 SQL ---
    print("正在生成 SQL 檔案...")
    sql_lines = []
    sql_lines.append("-- 自動生成的 D1 匯入腳本")
    sql_lines.append("BEGIN TRANSACTION;")
    sql_lines.append("")
    sql_lines.append("-- 1. 清理舊資料 (依需求選用)")
    sql_lines.append("DELETE FROM prompts;")
    sql_lines.append("DELETE FROM categories;")
    sql_lines.append("DELETE FROM sqlite_sequence WHERE name='categories' OR name='prompts';")
    sql_lines.append("")
    
    sql_lines.append("-- 2. 匯入 Categories")
    for cat in categories:
        # 跳脫單引號
        slug = cat['slug'].replace("'", "''")
        label = cat['label'].replace("'", "''")
        sql = f"INSERT INTO categories (slug, label, type, min_count, max_count, sort_order) VALUES ('{slug}', '{label}', '{cat['type']}', {cat['min_count']}, {cat['max_count']}, {cat['sort_order']});"
        sql_lines.append(sql)
        
    sql_lines.append("")
    sql_lines.append("-- 3. 匯入 Prompts")
    for p in prompts:
        cat_slug = p['category_slug'].replace("'", "''")
        p_label = p['label'].replace("'", "''")
        p_value = p['value'].replace("'", "''")
        sql = f"INSERT INTO prompts (category_slug, value, label, is_active) VALUES ('{cat_slug}', '{p_value}', '{p_label}', 1);"
        sql_lines.append(sql)
        
    sql_lines.append("")
    sql_lines.append("COMMIT;")
    
    # 寫入檔案
    output_filename = "import_data.sql"
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write("\n".join(sql_lines))
        
    print(f"完成！已生成 '{output_filename}'")
    print(f"共匯出: {len(categories)} 個類別, {len(prompts)} 個提示詞。")

if __name__ == "__main__":
    parse_and_convert()

正在解析設定...
正在解析內容...
正在生成 SQL 檔案...
完成！已生成 'import_data.sql'
共匯出: 27 個類別, 1114 個提示詞。
